In [23]:
#setup Odoo

import requests

jsonrpcid = 0
url = "https://demo-28-08-6.adhoc.ar/jsonrpc"
db = "demo-28-08-6"
username = "Chargebee"
password = "Chargebee"
headers = {'Content-Type': 'application/json'}


class Odoo():

    def __init__(self):
        self.jsonrpcid = 0

    def call(self, method: str, service: str, *args):
        self.jsonrpcid += 1

        data = {
            "jsonrpc": "2.0",
            "method": "call",
            "params": {
                "service": service,
                "method": method,
                "args": args
            },
            "id": self.jsonrpcid
        }

        # Realizar solicitud HTTP POST
        response = requests.post(url, headers=headers, json=data)
        result = response.json()

        return result

    def login(self, db: str, username: str, password: str):
        self.db = db
        self.username = username
        self.password = password
        outcome = self.call('login', 'common', self.db, self.username, self.password)
        if 'result' not in outcome:
            raise Exception('Failed to login')
        self.uid = outcome['result']

    def execute(self, *args):
        result = self.call('execute_kw', 'object', self.db, self.uid, self.password, *args)
        r = result.get('result', False)
        if not r:
            print(result)
            raise Exception('Command has failed')
        return r

    def read(self, model: str, ids: list=[], fields: list=[], context: dict={}):
        return self.execute(model, 'read', [ids, fields], {'context': context})

    def search_read(self, model: str, domain: list=[], fields: list=[], offset: int=0, limit: int=10, context: dict={}):
        return self.execute(model, 'search_read', [domain, fields, offset, limit], {'context': context})

    def create(self, model: str, vals_list: list=[], context: dict={}):
        return self.execute(model, 'create', vals_list, {'context': context})

# Crear nueva instancia
odoo = Odoo()

# Realizar login
odoo.login(db, username, password)

In [2]:
#setup chargebee

import chargebee

chargebee.configure('live_3NEiAmTg7dQArgRHR04LvHaIABupcuQOS', 'cliengo')

In [3]:
# Obtener las invoices desde Chargebee

import pandas as pd
import time
def get_chargebee_invoices_august_2024():
    invoice_list = []
    offset = None  # Inicialmente sin offset para la paginación
    start_date = int(time.mktime(time.strptime("2024-08-01", "%Y-%m-%d")))  # 1 de agosto de 2024 en Unix Timestamp
    end_date = int(time.mktime(time.strptime("2024-08-31 23:59:59", "%Y-%m-%d %H:%M:%S")))  # 31 de agosto de 2024

    while True:
        # Obtener la lista de facturas, sin límite explícito, pero con paginación
        entries = chargebee.Invoice.list({
            "status[in]": ["paid", "payment_due"],
            "date[between]": [start_date, end_date],  # Filtro por rango de fechas en Unix Timestamp
            "sort_by[asc]": "date",
            "limit": 100,  # Procesar 100 facturas por página
            "offset": offset  # Usar el offset actual para paginación
        })

        # Agregar las facturas a la lista
        for entry in entries:
            invoice = entry.invoice
            invoice_list.append(invoice.__dict__)  # Agregar la factura a la lista

        # Verificar si hay más páginas (si existe un offset para la siguiente página)
        offset = entries.next_offset
        if not offset:
            break  # Salir del bucle si no hay más páginas

    return invoice_list

# Llamar la función para obtener todas las facturas de agosto de 2024
invoices = get_chargebee_invoices_august_2024()

# Convertir la lista de diccionarios en un DataFrame de pandas
df = pd.DataFrame(invoices)

# Mostrar el DataFrame
df

,values,sub_types,dependant_types,id,po_number,customer_id,subscription_id,recurring,status,vat_number,...,void_reason_code,deleted,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,30025707,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,969726408,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2468,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2469,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2470,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,None,...,None,False,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [4]:
#para separar los campos dentro de jsons y diccionarios genere el siguiente codigo @Yepes
def print_column_types(dataframe):
    for column in dataframe.columns:
        dtype = dataframe[column].dtype
        print(f"{column} \t {dtype}")

# Llamar a la función para mostrar las columnas y tipos de datos
print_column_types(df)

values 	 object
sub_types 	 object
dependant_types 	 object
id 	 object
po_number 	 object
customer_id 	 object
subscription_id 	 object
recurring 	 bool
status 	 object
vat_number 	 object
price_type 	 object
date 	 int64
due_date 	 int64
net_term_days 	 int64
exchange_rate 	 float64
currency_code 	 object
total 	 int64
amount_paid 	 int64
amount_adjusted 	 int64
write_off_amount 	 int64
credits_applied 	 int64
amount_due 	 int64
paid_at 	 float64
dunning_status 	 object
next_retry_at 	 float64
voided_at 	 object
resource_version 	 int64
updated_at 	 int64
sub_total 	 int64
sub_total_in_local_currency 	 object
total_in_local_currency 	 object
local_currency_code 	 object
tax 	 int64
local_currency_exchange_rate 	 object
first_invoice 	 bool
new_sales_amount 	 float64
has_advance_charges 	 bool
term_finalized 	 bool
is_gifted 	 bool
generated_at 	 int64
expected_payment_date 	 float64
amount_to_collect 	 int64
round_off_amount 	 int64
line_items 	 object
discounts 	 object
line_item_di

In [5]:
#muestra todos los campos de tipo object ya que entre ellos estan esos campos a desanidar
df_objects = df.select_dtypes(include=['object'])
df_objects

,values,sub_types,dependant_types,id,po_number,customer_id,subscription_id,status,vat_number,price_type,...,payment_owner,void_reason_code,tax_category,vat_number_prefix,channel,business_entity_id,site_details_at_creation,tax_origin,object,base_currency_code
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,paid,30025707,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...,{},185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,paid,969726408,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2468,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2469,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD
2470,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...,{},187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,paid,None,tax_exclusive,...,None,None,None,None,web,None,"{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}",invoice,USD


In [6]:
#aqui separe especificamente estos 2 campos debido a que unos son json y otros son diccionarios, en este caso son 
dict_df = df_objects[['values', 'sub_types']]

In [7]:
dict_df

,values,sub_types
0,"{'id': '185392', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
1,"{'id': '185393', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
2,"{'id': '185394', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
3,"{'id': '185395', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
4,"{'id': '185396', 'po_number': 'Chile', 'custom...",{'line_items': <class 'chargebee.models.invoic...
...,...,...
2467,"{'id': '187908', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
2468,"{'id': '187910', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
2469,"{'id': '187892', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...
2470,"{'id': '187893', 'po_number': 'Argentina - Tra...",{'line_items': <class 'chargebee.models.invoic...


In [9]:
#y lo que aplico a este otro dataframe es la logica de separacion para json, que estos estaban raros por ese \n
dict_df2 = df_objects[['dependant_types', 'line_items', 'discounts', 'line_item_tiers', 'linked_payments',
'dunning_attempts', 'applied_credits', 'adjustment_credit_notes', 'issued_credit_notes', 'linked_orders', 'billing_address',
'site_details_at_creation', 'tax_origin']]

In [10]:
dict_df2

,dependant_types,line_items,discounts,line_item_tiers,linked_payments,dunning_attempts,applied_credits,adjustment_credit_notes,issued_credit_notes,linked_orders,billing_address,site_details_at_creation,tax_origin
0,{},"[{\n ""id"": ""li_6olL2UKBV7cg22wd"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_AzqB1XUMIWXep30Ov"",\n ...","[{\n ""created_at"": 1722567797,\n ""attemp...",[],[],[],[],"{\n ""company"": ""PARRA E HIJOS S.A."",\n ""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
1,{},"[{\n ""id"": ""li_Azz6aoUKC1bN11M2p"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16CXuLULLoKeC2h8c"",\n ...","[{\n ""created_at"": 1722568032,\n ""attemp...",[],[],[],[],"{\n ""company"": ""Nuke"",\n ""country"": ""AR""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2,{},"[{\n ""id"": ""li_AzyjxKUKC78gO1LOQ"",\n ""da...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16BW6dUJLpJrc1Cpj"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Martin"",\n ""last_name...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
3,{},"[{\n ""id"": ""li_6olL2UKCAuip39NE"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_AzqBCbUKy7RvD2RIN"",\n ...","[{\n ""created_at"": 1722567951,\n ""attemp...",[],[],[],[],"{\n ""company"": ""http://www.perkinelmer.com....","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
4,{},"[{\n ""id"": ""li_6olL2UKCDiUk3E1u"",\n ""dat...",None,"[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_6olL2UKCDiWS3E1z"",\n ...",[],[],[],[],[],"{\n ""first_name"": ""Francisco"",\n ""last_n...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,{},"[{\n ""id"": ""li_16BdL2UN5VwgXKJF5"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],[],[],"{\n ""company"": ""Renault Directa"",\n ""cou...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2468,{},"[{\n ""id"": ""li_16BdL2UN5Vwo1KJFY"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],[],[],"{\n ""company"": ""tuprode.com"",\n ""country...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2469,{},"[{\n ""id"": ""li_16BdL2UN5VvwOKJC3"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...",[],[],[],[],"[{\n ""cn_id"": ""CN-7066"",\n ""cn_reason_co...",[],"{\n ""first_name"": ""Mica"",\n ""last_name"":...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"
2470,{},"[{\n ""id"": ""li_16BdL2UN5VvzbKJCO"",\n ""da...","[{\n ""object"": ""discount"",\n ""entity_typ...","[{\n ""starting_unit"": 1,\n ""ending_unit""...","[{\n ""txn_id"": ""txn_16CXx1UO5eUVR3dvn"",\n ...","[{\n ""created_at"": 1725246813,\n ""attemp...",[],[],[],[],"{\n ""first_name"": ""Lucas"",\n ""last_name""...","{\n ""timezone"": ""America/Argentina/Buenos_A...","{\n ""country"": ""US""\n}"


In [11]:
#aqui estan las funciones para dividir los campos de diccionario

# Función para desanidar el diccionario en cada fila de una columna específica
def flatten_json_column(df, column_name):
    # Verificar si la columna es de tipo diccionario o tiene JSON
    if df[column_name].apply(lambda x: isinstance(x, dict)).all():
        # Desanidar la columna 'column_name'
        df_expanded = pd.json_normalize(df[column_name])
        
        # Concatenar las nuevas columnas con el DataFrame original, eliminando la columna original desanidada
        # Agregar sufijos a las columnas que se solapan.
        df = df.drop(columns=[column_name]).join(df_expanded, rsuffix=f'_{column_name}_expanded')
    
    return df

# Función para desanidar todas las columnas de tipo diccionario o JSON en un DataFrame
def flatten_all_json_columns(df):
    # Usar .map en lugar de .applymap para verificar columnas con diccionarios
    columns_to_flatten = df.columns[df.apply(lambda col: col.map(lambda x: isinstance(x, dict)).any())]
    for column in columns_to_flatten:
        df = flatten_json_column(df, column)
    
    return df

# Aplica la función a todas las columnas que contienen diccionarios o JSON
df_flattened_all = flatten_all_json_columns(dict_df)

# Imprimir el nuevo DataFrame con todas las columnas JSON desanidadas
df_flattened_all


,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,adjustment_credit_notes_sub_types_expanded,issued_credit_notes_sub_types_expanded,linked_orders_sub_types_expanded,notes_sub_types_expanded,shipping_address,statement_descriptor,billing_address,einvoice,site_details_at_creation,tax_origin
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,1722499680,1722499680,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,1722507420,1722507420,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,1722508740,1722508740,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,1722509640,1722509640,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,1722510308,1722510308,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee.models.invoice.Invoice.State...,<class 'chargebee.models.invoice.Invoice.Billi...,<class 'chargebee.models.invoice.Invoice.Einvo...,<class 'chargebee.models.invoice.Invoice.SiteD...,<class 'chargebee.models.invoice.Invoice.TaxOr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,tax_exclusive,1725159600,1725159600,0,...,<class 'chargebee.models.invoice.Invoice.Adjus...,<class 'chargebee.models.invoice.Invoice.Issue...,<class 'chargebee.models.invoice.Invoice.Linke...,<class 'chargebee.models.invoice.Invoice.Note'>,<class 'chargebee.models.invoice.Invoice.Shipp...,<class 'chargebee

In [12]:
#y aqui las columnas que dividen los campos json
import json

# Función para desanidar columnas que contienen listas de diccionarios (JSON)
def flatten_list_column(df, column_name):
    # Verificamos si las filas de la columna contienen listas de diccionarios
    df[column_name] = df[column_name].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    # Expandir la lista de diccionarios en nuevas filas
    df_expanded = df[column_name].explode().reset_index(drop=True)
    
    # Desanidar los diccionarios en nuevas columnas
    df_expanded = pd.json_normalize(df_expanded)
    
    # Concatenar el dataframe original con las columnas desanidadas
    df = df.drop(columns=[column_name]).join(df_expanded, rsuffix=f'_{column_name}_expanded')
    
    return df

# Función para desanidar todas las columnas que contienen listas de diccionarios en un DataFrame
def flatten_all_list_columns(df):
    # Verificar si alguna columna contiene listas de diccionarios
    columns_to_flatten = df.columns[df.apply(lambda col: col.map(lambda x: isinstance(x, list) and isinstance(x[0], dict) if isinstance(x, list) and len(x) > 0 else False).any())]
    
    # Desanidar cada columna que contenga listas de diccionarios
    for column in columns_to_flatten:
        df = flatten_list_column(df, column)
    
    return df

# Supongamos que tu dataframe se llama df_flattened_all, ya que es el resultado de desanidar los diccionarios.
# Ahora desanidamos las columnas que contienen listas de objetos (JSON).
df_final = flatten_all_list_columns(df_flattened_all)

# Mostrar el DataFrame con las listas desanidadas
df_final

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,1722499680,1722499680,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,1722507420,1722507420,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,1722508740,1722508740,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,1722509640,1722509640,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,1722510308,1722510308,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,tax_exclusive,1725159600,1725159600,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,1725159600,1725159600,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2469,187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,True,paid,tax_exclusive,1725159600,1725159600,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2470,187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,tax_exclusive,1725159600,1725159600,0,...,100.0,line_item_discount,li_16BdL2UN4mcBRIeat,item_level_coupon,700.0,MIGRA2024STARTER1USER,MIGRA2024STARTER1USER,NaN,NaN,NaN


In [13]:
#Le cambie la region a argentina utc tal y como lo tuvimos que hacer con panella cuando los traiamos de BigQuery antes
import pytz
argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')
df_final['date_from'] = pd.to_datetime(df_final['date_from'], unit='s', utc=True).dt.tz_convert(argentina_tz).dt.date
df_final['date_to'] = pd.to_datetime(df_final['date_to'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['applied_at'] = pd.to_datetime(df_final['applied_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['generated_at'] = pd.to_datetime(df_final['generated_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['updated_at'] = pd.to_datetime(df_final['updated_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['paid_at'] = pd.to_datetime(df_final['paid_at'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['due_date'] = pd.to_datetime(df_final['due_date'], unit='s', utc=True).dt.tz_convert(argentina_tz)
df_final['date'] = pd.to_datetime(df_final['date'], unit='s', utc=True).dt.tz_convert(argentina_tz)

In [14]:
df_final

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,187908,Argentina - Transferencia Bancaria (Leadaki),5f3da9f3b68252002aaac99e,AzZsu3UBeV4iY8HEC,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2469,187892,Argentina - Transferencia Bancaria (Leadaki),57bddbafe4b08f5c877fe039,169v0eUBcr4Bg73Qq,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2470,187893,Argentina - Transferencia Bancaria (Leadaki),5d3773fae4b0c496f93c2467,16CVC3UBe8bt98ESR,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,line_item_discount,li_16BdL2UN4mcBRIeat,item_level_coupon,700.0,MIGRA2024STARTER1USER,MIGRA2024STARTER1USER,NaN,NaN,NaN


In [15]:
#ordernar por id de manera descendente
order = df_final.sort_values(by = 'id', ascending=True)
order

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2462,187911,Argentina - Transferencia Bancaria (Leadaki),5727680de4b06a6e4222f0c8,AzyhzvUBeSCea8nZZ,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2463,187912,Argentina - Transferencia Bancaria (Leadaki),53c42a63e4b08c5d83b878ff,AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2455,187913,Argentina - Transferencia Bancaria (Leadaki),536109b2e4b0ae650c483374,16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
print_column_types(order)

id 	 object
po_number 	 object
customer_id 	 object
subscription_id 	 object
recurring 	 bool
status 	 object
price_type 	 object
date 	 datetime64[ns, America/Argentina/Buenos_Aires]
due_date 	 datetime64[ns, America/Argentina/Buenos_Aires]
net_term_days 	 int64
exchange_rate 	 float64
total 	 int64
amount_paid 	 int64
amount_adjusted 	 int64
write_off_amount 	 int64
credits_applied 	 int64
amount_due 	 int64
paid_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
dunning_status 	 object
updated_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
resource_version 	 int64
deleted 	 bool
object 	 object
first_invoice 	 bool
amount_to_collect 	 int64
round_off_amount 	 int64
has_advance_charges 	 bool
currency_code 	 object
base_currency_code 	 object
generated_at 	 datetime64[ns, America/Argentina/Buenos_Aires]
is_gifted 	 bool
term_finalized 	 bool
channel 	 object
tax 	 int64
sub_total 	 int64
linked_orders 	 object
billing_address.company 	 object
billing_address.country 	 object
bi

In [ ]:
#a partir de aqui empieza lo de Odoo

In [17]:
#primero con las pruebas de insercion en sales.order


odoo.create('sale.order', [{
                            'campaign_id': False,
                            'source_id': False,
                            'medium_id': False,
                            'activity_ids': [],
                            'activity_state': True,
                            'partner_id': 11, 
                            'company_id': 1,
                            'website_id': 1,
                            }])


356

In [17]:
# aqui yo buscaba antes un id recien creado por el codigo anterior
sale_order_345 = odoo.execute('sale.order', 'search_read', [[['id', '=', 354]], [], 0, 1])
print(sale_order_345)

[{'id': 354, 'campaign_id': False, 'source_id': False, 'medium_id': False, 'activity_ids': [], 'activity_state': False, 'activity_user_id': False, 'activity_type_id': False, 'activity_type_icon': False, 'activity_date_deadline': False, 'my_activity_date_deadline': False, 'activity_summary': False, 'activity_exception_decoration': False, 'activity_exception_icon': False, 'activity_calendar_event_id': False, 'message_is_follower': True, 'message_follower_ids': [1243], 'message_partner_ids': [121], 'message_ids': [2745], 'has_message': True, 'message_needaction': False, 'message_needaction_counter': 0, 'message_has_error': False, 'message_has_error_counter': 0, 'message_attachment_count': 0, 'rating_ids': [], 'website_message_ids': [], 'message_has_sms_error': False, 'access_url': '/my/orders/354', 'access_token': False, 'access_warning': '', 'name': 'V 0001-00000342', 'company_id': [1, 'REGEX LLC'], 'partner_id': [11, 'Gemini Furniture'], 'state': 'draft', 'locked': False, 'client_order_

In [17]:
#aqui hacemos el llamado a las invoices de clientes en Odoo

#odf = odoo.execute('account.move', 'search_read', [[['id', '>', 1]], 0, 20])
#dfc = pd.DataFrame(odf)
#dfc

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
0,47512,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
1,47511,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2,47510,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
3,47509,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
4,47508,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2278,19,False,False,False,False,False,BNK1/2024/,3,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2279,18,False,False,False,False,False,BNK1/2024/,2,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2280,17,False,False,False,False,False,BNK1/2024/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2281,11,False,False,False,False,False,RBILL/2023/05/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [18]:
#este nada mas es una busqueda de una factura en especifico por medio del campo name, 
# que es el que aparece en la interfaz grafica de odoo


#dfc.loc[dfc['name'] == 'BNK1/2024/00007']

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
2274,23,False,False,False,False,False,BNK1/2024/,7,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [19]:
#de la funcion que cree al principio se la aplique al dataframe de las invoices de odoo 
# para ver el tipo de datos de cada campo del modelo account.move


#print_column_types(dfc)

id 	 int64
campaign_id 	 bool
source_id 	 bool
medium_id 	 bool
is_tax_computed_externally 	 bool
is_avatax 	 bool
sequence_prefix 	 object
sequence_number 	 int64
activity_ids 	 object
activity_state 	 object
activity_user_id 	 object
activity_type_id 	 object
activity_type_icon 	 object
activity_date_deadline 	 object
my_activity_date_deadline 	 bool
activity_summary 	 object
activity_exception_decoration 	 bool
activity_exception_icon 	 bool
activity_calendar_event_id 	 bool
access_url 	 object
access_token 	 bool
access_warning 	 object
message_is_follower 	 bool
message_follower_ids 	 object
message_partner_ids 	 object
message_ids 	 object
has_message 	 bool
message_needaction 	 bool
message_needaction_counter 	 int64
message_has_error 	 bool
message_has_error_counter 	 int64
message_attachment_count 	 int64
rating_ids 	 object
website_message_ids 	 object
message_has_sms_error 	 bool
message_main_attachment_id 	 object
name 	 object
ref 	 object
date 	 object
state 	 object
move

In [20]:
#aqui hice el llamado de los campos de tipo object del dataframe para ver los campos que ocupan el equivalente a un valor de moneda
#lo hice para ir viendo los campos que mas adelante debo comparar con los de Chargebee y hacer la migracion correctamente


#floats = dfc.select_dtypes(include=['float64'])
#floats

,amount_untaxed,amount_tax,amount_total,amount_residual,amount_untaxed_signed,amount_tax_signed,amount_total_signed,amount_total_in_currency_signed,amount_residual_signed,quick_edit_total_amount,partner_credit,amount_paid,asset_remaining_value,asset_depreciated_value,depreciation_value,l10n_ar_currency_rate,computed_currency_rate
0,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0
1,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0
2,143.75,0.00,143.75,143.75,143.75,0.00,143.75,143.75,143.75,0.0,143.75,0.0,0.0,0.0,0.0,0.0,1.0
3,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0
4,86.25,0.00,86.25,86.25,86.25,0.00,86.25,86.25,86.25,0.0,86.25,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2278,0.00,0.00,32.58,0.00,0.00,0.00,32.58,32.58,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0
2279,0.00,0.00,1275.00,0.00,0.00,0.00,1275.00,1275.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0
2280,0.00,0.00,5103.00,0.00,0.00,0.00,5103.00,5103.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0
2281,541.10,81.17,622.27,622.27,541.10,81.17,622.27,622.27,622.27,0.0,622.27,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
#este codigo me muestra en un dataframe solamente los campos de tipo float de account.move pero sin excluir el id de la misma
# por eso le hago un concat con el original para que conserve el id

#if 'id' in dfc.columns:
#    floats_with_id = pd.concat([floats, dfc[['id', 'name']]], axis=1)
#else:
#    print("La columna 'id' no existe en el DataFrame original.")

#floats_with_id

,amount_untaxed,amount_tax,amount_total,amount_residual,amount_untaxed_signed,amount_tax_signed,amount_total_signed,amount_total_in_currency_signed,amount_residual_signed,quick_edit_total_amount,partner_credit,amount_paid,asset_remaining_value,asset_depreciated_value,depreciation_value,l10n_ar_currency_rate,computed_currency_rate,id,name
0,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0,47512,/
1,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0,47511,/
2,143.75,0.00,143.75,143.75,143.75,0.00,143.75,143.75,143.75,0.0,143.75,0.0,0.0,0.0,0.0,0.0,1.0,47510,/
3,28.75,0.00,28.75,28.75,28.75,0.00,28.75,28.75,28.75,0.0,28.75,0.0,0.0,0.0,0.0,0.0,1.0,47509,/
4,86.25,0.00,86.25,86.25,86.25,0.00,86.25,86.25,86.25,0.0,86.25,0.0,0.0,0.0,0.0,0.0,1.0,47508,/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2278,0.00,0.00,32.58,0.00,0.00,0.00,32.58,32.58,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,19,BNK1/2024/00003
2279,0.00,0.00,1275.00,0.00,0.00,0.00,1275.00,1275.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,18,BNK1/2024/00002
2280,0.00,0.00,5103.00,0.00,0.00,0.00,5103.00,5103.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.0,17,BNK1/2024/00001
2281,541.10,81.17,622.27,622.27,541.10,81.17,622.27,622.27,622.27,0.0,622.27,0.0,0.0,0.0,0.0,0.0,1.0,11,RBILL/2023/05/0001


In [22]:
#aqui fue en donde vi como es que se almacena el registro de partner_id (que es un array)

#sub_dfc = dfc['partner_id']
#sub_dfc

0       [30787, 66b5611d83457860cafb49eb]
1       [30729, 66b5408baa08501f9aa2503a]
2       [30420, 66b3eb155ac43769d434ca95]
3       [29370, 66b2e905e699225d840d64f8]
4       [30390, 66b28d2b796a601e95fe096d]
                      ...                
2278                                False
2279                 [14, Azure Interior]
2280                                False
2281                 [14, Azure Interior]
2282                 [14, Azure Interior]
Name: partner_id, Length: 2283, dtype: object

In [17]:
# Leer todas las facturas (sin establecer un dominio específico) todo para comprobar que la insercion fue exitosa
invoices = odoo.search_read('account.move', [], ['id', 'name', 'date', 'amount_total', 'partner_id'])

customers_invoice = pd.DataFrame(invoices)
customers_invoice


,id,name,date,amount_total,partner_id
0,66062,/,2024-10-09,1081.00,"[108, Ayuntamiento De Bilbao, Departamento de ..."
1,164,/,2024-09-20,28.75,"[11, Gemini Furniture]"
2,161,INV/2024/00011,2024-09-17,28.75,"[11, Gemini Furniture]"
3,163,/,2024-09-17,0.00,False
4,160,/,2024-09-17,0.00,"[11, Gemini Furniture]"
5,159,/,2024-09-16,28.75,"[11, Gemini Furniture]"
6,158,/,2024-09-16,28.75,"[11, Gemini Furniture]"
7,157,/,2024-09-16,0.00,"[11, Gemini Furniture]"
8,162,INV/2024/00012,2024-09-10,575.00,"[10, Deco Addict]"
9,50,/,2024-08-31,0.00,False


In [18]:
#llame a todas las invoice otra vez pero para separar los campos del array(al final no sirvio porque solamente fue una operacion 
# de lectura :(  )
i = odoo.search_read('account.move', [], [], limit=None)
inv = pd.DataFrame(i)
inv

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
0,66062,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
1,164,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2,161,False,False,False,False,False,INV/2024/,11,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
3,163,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
4,160,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,19,False,False,False,False,False,BNK1/2024/,3,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2417,18,False,False,False,False,False,BNK1/2024/,2,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2418,17,False,False,False,False,False,BNK1/2024/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2419,11,False,False,False,False,False,RBILL/2023/05/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [19]:
#solo mostrar el campo de partner_id para ver como quedan los customer_id de Chargebee  en Odoo
inv[['id', 'partner_id', 'name', 'invoice_date', 'invoice_line_ids']]

,id,partner_id,name,invoice_date,invoice_line_ids
0,66062,"[108, Ayuntamiento De Bilbao, Departamento de ...",/,False,"[68586, 68592]"
1,164,"[11, Gemini Furniture]",/,False,[]
2,161,"[11, Gemini Furniture]",INV/2024/00011,2024-09-17,[473]
3,163,False,/,False,[479]
4,160,"[11, Gemini Furniture]",/,False,[]
...,...,...,...,...,...
2416,19,False,BNK1/2024/00003,False,"[57, 58]"
2417,18,"[14, Azure Interior]",BNK1/2024/00002,False,"[55, 56]"
2418,17,False,BNK1/2024/00001,False,"[53, 54]"
2419,11,"[14, Azure Interior]",RBILL/2023/05/0001,2023-05-19,[20]


In [20]:
#realice el llamado del campo del customer de Chargebee para ingresarlo en res.partner
customerdf = order['customer_id']
customerdf

0       5acdf7d8e4b014cba3b1724b
1       5aa291c2e4b0c65fcc798ce8
2       5ad08c66e4b01bf0eefaf887
3       5cb4d5d0e4b0bd6bed7869fa
4       5b47de04e4b03c859495a0f7
                  ...           
2468    5a71ba83e4b07b95e31fc272
2462    5727680de4b06a6e4222f0c8
2463    53c42a63e4b08c5d83b878ff
2455    536109b2e4b0ae650c483374
2471    5a0c58c9e4b031a2c96d319c
Name: customer_id, Length: 2472, dtype: object

In [21]:
#como los customers que vamos a ingresar estaban duplicados en el llamado, pues elimine los duplicados ya que a partir de este dataframe 
#hare las inserciones en el modelo res.partner
customers = customerdf.drop_duplicates()
customers

0       5acdf7d8e4b014cba3b1724b
1       5aa291c2e4b0c65fcc798ce8
2       5ad08c66e4b01bf0eefaf887
3       5cb4d5d0e4b0bd6bed7869fa
4       5b47de04e4b03c859495a0f7
                  ...           
2468    5a71ba83e4b07b95e31fc272
2462    5727680de4b06a6e4222f0c8
2463    53c42a63e4b08c5d83b878ff
2455    536109b2e4b0ae650c483374
2471    5a0c58c9e4b031a2c96d319c
Name: customer_id, Length: 2262, dtype: object

In [24]:
#busque los res.partner que es en donde se van a almacenar los customer de Chargebee

i2 = odoo.search_read('res.partner', [], [], limit=None)  # Establecer limit=None para obtener todos los registros
partner = pd.DataFrame(i2)

partner

,id,avatax_unique_code,website_id,website_published,is_published,can_publish,website_url,message_is_follower,message_follower_ids,message_partner_ids,...,l10n_es_edi_facturae_ac_role_type_ids,l10n_es_edi_facturae_ac_physical_gln,l10n_es_edi_facturae_ac_logical_operational_point,sale_type,subscription_count,last_website_so_id,l10n_cl_delivery_guide_price,arba_alicuot_ids,drei,default_regimen_ganancias_id
0,86,Contact 86,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1,87,Contact 87,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2,67,Contact 67,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
3,34827,Contact 34827,False,False,False,True,#,True,[99382],[121],...,[],False,False,False,0,False,sale_order,[],False,False
4,35560,Contact 35560,False,False,False,True,#,True,[100115],[121],...,[],False,False,False,0,False,sale_order,[],False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,69,Contact 69,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2379,77,Contact 77,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2380,63,Contact 63,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2381,76,Contact 76,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False


In [25]:
#deshacerme de los nombres duplicados del modelo res.partner
all_partners=partner.drop_duplicates('name')
all_partners

,id,avatax_unique_code,website_id,website_published,is_published,can_publish,website_url,message_is_follower,message_follower_ids,message_partner_ids,...,l10n_es_edi_facturae_ac_role_type_ids,l10n_es_edi_facturae_ac_physical_gln,l10n_es_edi_facturae_ac_logical_operational_point,sale_type,subscription_count,last_website_so_id,l10n_cl_delivery_guide_price,arba_alicuot_ids,drei,default_regimen_ganancias_id
0,86,Contact 86,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
1,87,Contact 87,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2,67,Contact 67,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
3,34827,Contact 34827,False,False,False,True,#,True,[99382],[121],...,[],False,False,False,0,False,sale_order,[],False,False
4,35560,Contact 35560,False,False,False,True,#,True,[100115],[121],...,[],False,False,False,0,False,sale_order,[],False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,69,Contact 69,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2379,77,Contact 77,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2380,63,Contact 63,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False
2381,76,Contact 76,False,False,False,True,#,False,[],[],...,[],False,False,False,0,False,sale_order,[],False,False


In [26]:
#visualizo el id y elcampo en donde se muestra el nombre del customer
i3 = partner[['name', 'id']]

In [27]:
i3

,name,id
0,(AR) Exento,86
1,(AR) Monotributista,87
2,0h3n-rijaune@yahoo.example.com,67
3,16CXyqUDJkqSi1S5N,34827
4,535680f7e4b03a7c45f66703,35560
...,...,...
2378,the.jose@gmail.example.com,69
2379,the.real.shane@gmail.example.com,77
2380,thing.thang.thong@gmail.example.com,63
2381,tina.turner@gmail.example.com,76


In [108]:
#luego ejecute una insercion en ese modelo y funciono exitosamente (el campo que vi que podia llenar con el string de un customer_id)
#era ese campo name de todos los que tiene el dataframe


#odoo.create('res.partner', [{
#    'name': '5dde8435e4b098a27bbd4def'
#}])

465

In [27]:
#veo los tipos de datos del dataframe de los partners solo con nombre y id
print_column_types(i3)

name 	 object
id 	 int64


In [225]:
#genere un dataframe sobre los campos con tipo de dato object que es un equivalente a un string y poder encontrar que campo habia que
# llenar para que me se pudiera ver el nombre del partner en la interfaz grafica


#many = partner.select_dtypes(include=['object'])
#many

,avatax_unique_code,website_url,message_follower_ids,message_partner_ids,message_ids,rating_ids,website_message_ids,email_normalized,activity_ids,image_1920,...,gross_income_jurisdiction_ids,imp_ganancias_padron,actividades_padron,impuestos_padron,l10n_cl_dte_email,l10n_es_edi_facturae_ac_center_code,l10n_es_edi_facturae_ac_role_type_ids,l10n_cl_delivery_guide_price,arba_alicuot_ids,default_regimen_ganancias_id
0,Contact 86,#,[],[],[1115],[],[],info@exento.example.com,[],iVBORw0KGgoAAAANSUhEUgAAAcIAAAB4CAYAAABhPvLiAA...,...,[],False,[],[],False,False,[],sale_order,[],False
1,Contact 87,#,[],[],"[1382, 1126]",[],[],info@example.com,[],iVBORw0KGgoAAAANSUhEUgAAAcIAAAB4CAYAAABhPvLiAA...,...,[],False,[],[],False,False,[],sale_order,[],False
2,Contact 67,#,[],[],[480],[],[],0h3n-rijaune@yahoo.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
3,Contact 34827,#,[99382],[121],[119352],[],[],False,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
4,Contact 35560,#,[100115],[121],[120085],[],[],False,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,Contact 69,#,[],[],[484],[],[],the.jose@gmail.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
2379,Contact 77,#,[],[],[500],[],[],the.real.shane@gmail.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
2380,Contact 63,#,[],[],[472],[],[],thing.thang.thong@gmail.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False
2381,Contact 76,#,[],[],[498],[],[],tina.turner@gmail.example.com,[],False,...,[],False,[],[],False,False,[],sale_order,[],False


In [226]:
#una vez encontrado lo llamo del dataframe anterior


#many['name']

0                               (AR) Exento
1                       (AR) Monotributista
2            0h3n-rijaune@yahoo.example.com
3                         16CXyqUDJkqSi1S5N
4                  535680f7e4b03a7c45f66703
                       ...                 
2378             the.jose@gmail.example.com
2379       the.real.shane@gmail.example.com
2380    thing.thang.thong@gmail.example.com
2381          tina.turner@gmail.example.com
2382             yin.lee@wechat.example.com
Name: name, Length: 2383, dtype: object

In [28]:
#empiezo a generar el codigo para hacer una insercion de los customers de Chargebee en Odoo
# Inicializar una lista para almacenar el historial de inserciones
insertion_history = []

# Función para insertar nuevos registros en res.partner
def insert_new_partners(customers, execution_id):
    # Leer todos los registros actuales del modelo res.partner
    existing_partners = odoo.search_read('res.partner', [], ['name'])
    
    # Extraer los nombres de los partners existentes
    existing_partner_names = {partner['name'] for partner in existing_partners}
    
    # Obtener la lista de nombres de customers del DataFrame
    customer_ids = customers.tolist()
    
    # Crear una lista de diccionarios para los nuevos registros a insertar
    # Solo insertar aquellos que no existen en el modelo res.partner
    records_to_insert = [{'name': customer_id} for customer_id in customer_ids if customer_id not in existing_partner_names]
    
    # Realizar la inserción masiva uno por uno para evitar sobrecarga en la llamada a Odoo.create
    for record in records_to_insert:
        odoo.create('res.partner', [record])
    
    # Guardar los nuevos registros en el historial con un ID de ejecución
    insertion_history.append({
        'execution_id': execution_id,
        'new_records': records_to_insert
    })
    
    # Mostrar cuántos registros nuevos fueron insertados
    print(f"Se insertaron {len(records_to_insert)} nuevos registros en res.partner.")


In [29]:

# Función para obtener los registros insertados en una ejecución específica
def get_inserted_records(execution_id):
    # Buscar la ejecución en el historial
    for execution in insertion_history:
        if execution['execution_id'] == execution_id:
            # Convertir la lista de diccionarios en un DataFrame
            df = pd.DataFrame(execution['new_records'])
            return df
    
    # Si no se encuentra la ejecución
    print(f"No se encontraron registros para la ejecución con ID: {execution_id}")
    return None

In [216]:
# Insertar nuevos registros con un ID de ejecución '1'
insert_new_partners(customers, execution_id='1')

Se insertaron 2265 nuevos registros en res.partner.


In [217]:
# Obtener los registros insertados en la ejecución '1'
#esto lo hago para tener el registro de que data acabo de importar para luego tener ese bloque de data para generar las invoices
# o incluso borrarlos cuando lleguemos a otras metodologias

df_execution_1 = get_inserted_records('1')
d_exec = pd.DataFrame(df_execution_1)
d_exec

,name
0,5acdf7d8e4b014cba3b1724b
1,5aa291c2e4b0c65fcc798ce8
2,5ad08c66e4b01bf0eefaf887
3,5cb4d5d0e4b0bd6bed7869fa
4,5b47de04e4b03c859495a0f7
...,...
2260,5a71ba83e4b07b95e31fc272
2261,5727680de4b06a6e4222f0c8
2262,53c42a63e4b08c5d83b878ff
2263,536109b2e4b0ae650c483374


In [32]:
#genere este codigo para las ocasiones en las que hay que realizar la eliminacion de muchos registros sin que afecte a los que hay que 
# proteger ya que al hacer el llamado directo y luego borrar es lo que podria sabotear el trabajo debido a un error humano jaja
# Obtener todos los registros del modelo res.partner, ordenados por id de forma descendente y con los campos 'id' y 'name'
i5 = odoo.search_read('res.partner', [], ['id', 'name'], limit=None)

# Convertir los resultados a un DataFrame
partner2 = pd.DataFrame(i5)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last = partner2.sort_values(by='id', ascending=False).head(2265)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_rows_df = pd.DataFrame(last)
last_rows_df

,id,name
179,35591,5a0c58c9e4b031a2c96d319c
5,35590,536109b2e4b0ae650c483374
6,35589,53c42a63e4b08c5d83b878ff
27,35588,5727680de4b06a6e4222f0c8
215,35587,5a71ba83e4b07b95e31fc272
...,...,...
345,33331,5b47de04e4b03c859495a0f7
561,33330,5cb4d5d0e4b0bd6bed7869fa
266,33329,5ad08c66e4b01bf0eefaf887
241,33328,5aa291c2e4b0c65fcc798ce8


In [33]:
#aqui llame al dataframe de los registros de res.partner para ver solamente el nombre y id
p_id = partner[['id', 'name']]
p_id_df = pd.DataFrame(p_id)
p_id_df

,id,name
0,86,(AR) Exento
1,87,(AR) Monotributista
2,67,0h3n-rijaune@yahoo.example.com
3,34827,16CXyqUDJkqSi1S5N
4,35560,535680f7e4b03a7c45f66703
...,...,...
2378,69,the.jose@gmail.example.com
2379,77,the.real.shane@gmail.example.com
2380,63,thing.thang.thong@gmail.example.com
2381,76,tina.turner@gmail.example.com


In [229]:
#despues uni el dataframe que me mostraba el resultado de los customers que agregue 
# para que hiciera una union con el dataframe de los partners ya que al ser exitosa la insercion de los customers en odoo
# ahora esos customers de chargeBee ya estan en Odoo



#new_partners_df = pd.merge(p_id_df, d_exec, on='name', how='inner')
#new_partners_df

In [230]:
#ahora voy a eliminar esos registros ya que aun no habia corregido los customers ya que hay duplicados


#partner_id_to_delete = 131  # Reemplazar con el ID real que hay que eliminar

# Realizar la operación de eliminación


#odoo.execute('res.partner', 'unlink', [partner_id_to_delete])


In [201]:
# Obtener los IDs de los últimos  registros
partner_ids_to_delete = last['id'].tolist()

# Eliminar los registros utilizando el método 'unlink' de Odoo
if partner_ids_to_delete:
    odoo.execute('res.partner', 'unlink', [partner_ids_to_delete])

# Mostrar el número de registros eliminados
print(f"Se han eliminado {len(partner_ids_to_delete)} registros de res.partner.")

Se han eliminado 2266 registros de res.partner.


In [37]:
#Buscar un registro de los recien ingresados partners por medio de su id

#last_rows_df.loc[last_rows_df['id'] == 25762]

,id,name


In [38]:
#Este codigo estaba de prueba para verficar que no se dupliquen los registros y el resultado fue exitoso

#dd = last_rows_df.drop_duplicates('name')
#dd

,id,name
179,33326,5a0c58c9e4b031a2c96d319c
5,33325,536109b2e4b0ae650c483374
6,33324,53c42a63e4b08c5d83b878ff
27,33323,5727680de4b06a6e4222f0c8
215,33322,5a71ba83e4b07b95e31fc272
...,...,...
345,31065,5b47de04e4b03c859495a0f7
561,31064,5cb4d5d0e4b0bd6bed7869fa
266,31063,5ad08c66e4b01bf0eefaf887
241,31062,5aa291c2e4b0c65fcc798ce8


In [34]:
#unir los dataframes que muestran los ultimos partneres con uno de llamado a los partners para confirmar que los partners coincidad 
# 
 
last_partners_df = pd.merge(p_id_df, last_rows_df, on='name', how='inner')
last_partners_df

,id_x,name,id_y
0,34827,16CXyqUDJkqSi1S5N,34827
1,35560,535680f7e4b03a7c45f66703,35560
2,35590,536109b2e4b0ae650c483374,35590
3,35589,53c42a63e4b08c5d83b878ff,35589
4,35586,540f3b9ee4b0492f27113fe7,35586
...,...,...,...
2260,35455,66ccdda3b1c5d119cf1a2412,35455
2261,35431,66cf6e434427b90002f38734,35431
2262,35428,66cf6f0374ecf760b7a5b258,35428
2263,35490,66d0a6efa9c6e736847a56e6,35490


In [35]:
#renombrar el campo name de res.partner para que se vea como el id del partner o de customer
r_last_partners_df = last_partners_df.rename(columns={'name': 'customer_id', 'id_y': 'partner_id'})
renamed_last_partners_df = r_last_partners_df.drop(columns=['id_x'])
renamed_last_partners_df

,customer_id,partner_id
0,16CXyqUDJkqSi1S5N,34827
1,535680f7e4b03a7c45f66703,35560
2,536109b2e4b0ae650c483374,35590
3,53c42a63e4b08c5d83b878ff,35589
4,540f3b9ee4b0492f27113fe7,35586
...,...,...
2260,66ccdda3b1c5d119cf1a2412,35455
2261,66cf6e434427b90002f38734,35431
2262,66cf6f0374ecf760b7a5b258,35428
2263,66d0a6efa9c6e736847a56e6,35490


In [36]:
#unir el dataframe anterior con el de order que son todas las invoices de Chargebee y que estos coincidad para luego insertar los
#datos de Chargebee necesarios para migrar a Odoo, terminando en un dataframe que tiene el partner_id y los campos de invoices de
#Chargebee, esto lo hago para que en un solo dataframe tenga toda la data necesaria para posteriormente introducirla en account.move


partners_Odoo_invoices = pd.merge(renamed_last_partners_df, order, on='customer_id', how='inner')
partners_Odoo_invoices

,customer_id,partner_id,id,po_number,subscription_id,recurring,status,price_type,date,due_date,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,16CXyqUDJkqSi1S5N,34827,187055,fertestoffline,AzZnUNUDJodaa1Uow,True,payment_due,tax_exclusive,2024-08-20 09:53:33-03:00,2024-08-20 09:53:33-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,535680f7e4b03a7c45f66703,35560,187881,Argentina - Transferencia Bancaria (Leadaki),AzqB5PUBczDZj76jS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,...,NaN,line_item_discount,li_16BdL2UN1xX73CrTF,item_level_coupon,2400.0,CTASPRUEBA-100%-FOREVER,CTASPRUEBA-100%-FOREVER,NaN,NaN,NaN
2,536109b2e4b0ae650c483374,35590,187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53c42a63e4b08c5d83b878ff,35589,187912,Argentina - Transferencia Bancaria (Leadaki),AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,540f3b9ee4b0492f27113fe7,35586,187909,Argentina - Transferencia Bancaria (Leadaki),AzyhzvUBeTc508oWU,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,66ccdda3b1c5d119cf1a2412,35455,187751,Argentina,16CV2TUMs0jeK371d,True,paid,tax_exclusive,2024-08-29 16:34:47-03:00,2024-08-29 16:34:47-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,66cf6e434427b90002f38734,35431,187720,Mexico,AzZnUfUMqw8ci2C2q,True,paid,tax_exclusive,2024-08-29 12:10:15-03:00,2024-08-29 12:10:15-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2469,66cf6f0374ecf760b7a5b258,35428,187717,Colombia,16CV2TUMqv4Dc2Ag8,True,paid,tax_exclusive,2024-08-29 12:06:00-03:00,2024-08-29 12:06:00-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2470,66d0a6efa9c6e736847a56e6,35490,187789,Cuenta test,16CXxzUMwehmC2IUg,True,paid,tax_exclusive,2024-08-30 11:38:38-03:00,2024-08-30 11:38:38-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
order

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2462,187911,Argentina - Transferencia Bancaria (Leadaki),5727680de4b06a6e4222f0c8,AzyhzvUBeSCea8nZZ,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2463,187912,Argentina - Transferencia Bancaria (Leadaki),53c42a63e4b08c5d83b878ff,AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2455,187913,Argentina - Transferencia Bancaria (Leadaki),536109b2e4b0ae650c483374,16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
#aqui lo escribo el codigo de insercion para que se vea que funciona al hacerla registro por registro

#aqui el codigo que genera una invoice de clientes 


#odoo.create('account.move', [{
 #   'move_type': 'out_invoice',  # Factura de cliente
  #  'partner_id': 138,  # ID del cliente
   # 'company_id': 1,  # ID de la compañía
    #'website_id': 1,  # ID del sitio web
    #'campaign_id': False,
    #'source_id': False,
    #'medium_id': False,
    #'activity_ids': [],
    #'amount_total': 28.75,
    #'amount_residual': 28.75,
    #'amount_total_signed': 28.75,
    #'amount_total_in_currency_signed': 28.75,
    #'amount_residual_signed': 28.75,
    #'activity_state': True,
#}])

166

In [55]:
odoo.create('account.move', [{
            'move_type': 'out_invoice',
            'partner_id': 34827,
            'journal_id': 1,
            'currency_id' : 1,
            'amount_untaxed': 0.00,
            'amount_untaxed_signed': 0.00,
            'amount_tax': 0.00,
            'amount_tax_signed': 0.00
    }])

66064

In [38]:
cur = odoo.search_read('account.move', [], ['currency_id'], limit=None)

# Convertir los resultados a un DataFrame
currency_c = pd.DataFrame(cur)
currency_c

,id,currency_id
0,66062,"[1, USD]"
1,164,"[1, USD]"
2,161,"[1, USD]"
3,163,"[1, USD]"
4,160,"[1, USD]"
...,...,...
2416,19,"[1, USD]"
2417,18,"[1, USD]"
2418,17,"[1, USD]"
2419,11,"[1, USD]"


In [238]:
# Revisar la estructura de los datos
print(currency_c['currency_id'].head(10))  # Revisar si los valores están en el formato correcto

0    [1, USD]
1    [1, USD]
2    [1, USD]
3    [1, USD]
4    [1, USD]
5    [1, USD]
6    [1, USD]
7    [1, USD]
8    [1, USD]
9    [1, USD]
Name: currency_id, dtype: object


In [39]:
# Asegurarse de que currency_id sean listas y cada lista tenga dos elementos
currency_c['currency_id'] = currency_c['currency_id'].apply(lambda x: x if isinstance(x, list) and len(x) == 2 else [None, None])

# Separar la columna currency_id en dos columnas: currency_num y currency_name
currency_c[['currency_num', 'currency_code']] = pd.DataFrame(currency_c['currency_id'].tolist(), index=currency_c.index)

# Convertir el campo currency_num a tipo numérico
currency_c['currency_num'] = pd.to_numeric(currency_c['currency_num'], errors='coerce')

# Verificar el resultado refinado con las nuevas columnas
currency_refined = currency_c[['currency_num', 'currency_code']]
currency_refined

,currency_num,currency_code
0,1,USD
1,1,USD
2,1,USD
3,1,USD
4,1,USD
...,...,...
2416,1,USD
2417,1,USD
2418,1,USD
2419,1,USD


In [40]:
currency_re_defined = currency_refined.drop_duplicates('currency_code')
currency_re_defined

,currency_num,currency_code
0,1,USD


In [41]:
partners_to_odoo_invoices = pd.merge(currency_re_defined, partners_Odoo_invoices, on='currency_code', how='inner')
partners_to_odoo_invoices

,currency_num,currency_code,customer_id,partner_id,id,po_number,subscription_id,recurring,status,price_type,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,1,USD,16CXyqUDJkqSi1S5N,34827,187055,fertestoffline,AzZnUNUDJodaa1Uow,True,payment_due,tax_exclusive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,USD,535680f7e4b03a7c45f66703,35560,187881,Argentina - Transferencia Bancaria (Leadaki),AzqB5PUBczDZj76jS,True,paid,tax_exclusive,...,NaN,line_item_discount,li_16BdL2UN1xX73CrTF,item_level_coupon,2400.0,CTASPRUEBA-100%-FOREVER,CTASPRUEBA-100%-FOREVER,NaN,NaN,NaN
2,1,USD,536109b2e4b0ae650c483374,35590,187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,USD,53c42a63e4b08c5d83b878ff,35589,187912,Argentina - Transferencia Bancaria (Leadaki),AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,USD,540f3b9ee4b0492f27113fe7,35586,187909,Argentina - Transferencia Bancaria (Leadaki),AzyhzvUBeTc508oWU,True,paid,tax_exclusive,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2465,1,USD,66ccdda3b1c5d119cf1a2412,35455,187751,Argentina,16CV2TUMs0jeK371d,True,paid,tax_exclusive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2466,1,USD,66cf6e434427b90002f38734,35431,187720,Mexico,AzZnUfUMqw8ci2C2q,True,paid,tax_exclusive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,1,USD,66cf6f0374ecf760b7a5b258,35428,187717,Colombia,16CV2TUMqv4Dc2Ag8,True,paid,tax_exclusive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,1,USD,66d0a6efa9c6e736847a56e6,35490,187789,Cuenta test,16CXxzUMwehmC2IUg,True,paid,tax_exclusive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
#visualizar si algun registro de facturas de agosto tiene impuestos

partners_to_odoo_invoices.loc[partners_to_odoo_invoices['tax_amount'] > 0]

,currency_num,currency_code,customer_id,partner_id,id,po_number,subscription_id,recurring,status,price_type,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded


In [170]:
#aqui llame a un dataframe que declare mas abajo, ignora esto

# date_df

In [43]:
partners_to_odoo_invoices['date_from']

0       2024-08-08
1       2024-08-13
2       2024-08-13
3       2024-08-13
4       2024-08-13
           ...    
2465    2024-08-12
2466    2024-08-12
2467    2024-08-12
2468    2024-08-12
2469    2024-08-12
Name: date_from, Length: 2470, dtype: object

In [44]:
# Filtrar las columnas que contienen 'date' en su nombre
date_columns = [col for col in order.columns if 'tax' in col.lower()]

# Crear un nuevo dataframe solo con las columnas que contienen 'date'
date_df = order[date_columns]

# Mostrar el nuevo dataframe filtrado
date_df


,tax,tax_origin.country,taxes,line_item_taxes,tax_origin,is_taxed,tax_amount,tax_exempt_reason
0,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
1,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
2,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
3,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
4,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
...,...,...,...,...,...,...,...,...
2468,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
2462,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
2463,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
2455,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured


In [45]:
# Filtrar las columnas que contienen 'date' en su nombre
tax_columns = [col for col in inv.columns if 'tax' in col.lower()]

# Crear un nuevo dataframe solo con las columnas que contienen 'date'
tax = inv[tax_columns]

# Mostrar el nuevo dataframe filtrado
tax


,is_tax_computed_externally,is_avatax,tax_cash_basis_rec_id,tax_cash_basis_origin_move_id,tax_cash_basis_created_move_ids,always_tax_exigible,tax_calculation_rounding_method,amount_untaxed,amount_tax,amount_untaxed_signed,...,tax_totals,tax_lock_date_message,tax_country_id,tax_country_code,avatax_unique_code,avatax_tax_date,tax_closing_end_date,tax_report_control_error,tax_closing_alert,tax_closing_show_multi_closing_warning
0,False,False,False,False,[],False,round_per_line,940.0,141.00,940.0,...,"{'amount_untaxed': 940.0, 'amount_total': 1081...",False,"[233, United States]",US,Journal Entry 66062,False,False,False,False,False
1,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 164,False,False,False,False,False
2,False,False,False,False,[],False,round_per_line,25.0,3.75,25.0,...,"{'amount_untaxed': 25.0, 'amount_total': 28.75...",False,"[233, United States]",US,Journal Entry 161,False,False,False,False,False
3,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 163,False,False,False,False,False
4,False,False,False,False,[],False,round_per_line,0.0,0.00,0.0,...,"{'amount_untaxed': 0.0, 'amount_total': 0.0, '...",False,"[233, United States]",US,Journal Entry 160,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2416,False,False,False,False,[],True,round_per_line,0.0,0.00,0.0,...,False,False,"[233, United States]",US,Journal Entry 19,False,False,False,False,False
2417,False,False,False,False,[],True,round_per_line,0.0,0.00,0.0,...,False,False,"[233, United States]",US,Journal Entry 18,False,False,False,False,False
2418,False,False,False,False,[],True,round_per_line,0.0,0.00,0.0,...,False,False,"[233, United States]",US,Journal Entry 17,False,False,False,False,False
2419,False,False,False,False,[],False,round_per_line,541.1,81.17,541.1,...,"{'amount_untaxed': 541.1, 'amount_total': 622....",False,"[233, United States]",US,Journal Entry 11,False,False,False,False,False


In [251]:
#una vez teniendo el modelo de res.partner e identificado los datos que debo insertar, generare la insercion de invoices en Odoo
#solamente como prueba ya que unicamente voy a incluir unos campos a las facturas pero lo mas importante en este caso es ingresar el
# partner_id de cada partner que ya enliste despues de ingresar el monton de registros de Chargebee en Odoo y asi de una vez saber a que
#partners debo ingresar en las invoices 
# Función para insertar facturas en account.move con validación de datos
import signal
import time

# Variable global para indicar si hubo una interrupción manual o del sistema
interrupted = False

# Función para manejar señales de interrupción (como CTRL+C o kill)
def handle_interrupt(signum, frame):
    global interrupted
    print(f"Señal {signum} recibida. Iniciando rollback...")
    interrupted = True  # Marcar interrupción
    raise Exception("Proceso interrumpido manualmente o por el sistema.")

# Conectar las señales de interrupción al manejador de interrupciones
signal.signal(signal.SIGINT, handle_interrupt)   # CTRL+C
signal.signal(signal.SIGTERM, handle_interrupt)  # kill command

def insert_invoices_with_rollback(partners_to_odoo_invoices, execution_id):
    global interrupted
    records_to_insert = []
    inserted_record_ids = []

    # Iterar sobre el DataFrame
    for _, row in partners_to_odoo_invoices.iterrows():
        record = {
            'move_type': 'out_invoice',
            'partner_id': row['partner_id'],
            'journal_id': 1,
            'currency_id': row['currency_num'],
            'date': row['date_from'].strftime('%Y-%m-%d'),
            'invoice_date': row['date_from'].strftime('%Y-%m-%d'),
            'amount_untaxed': row['tax_amount'],
            'amount_untaxed_signed': row['tax_amount'],
            'amount_tax': row['tax_amount'],
            'amount_tax_signed': row['tax_amount']
        }

        if 'partner_id' not in record or not record['partner_id']:
            print(f"Registro inválido: {record}")
            continue

        records_to_insert.append(record)

    if records_to_insert:
        try:
            # Realizar inserciones en un bucle
            for record in records_to_insert:
                if interrupted:
                    raise Exception("Interrupción detectada antes de insertar.")
                
                # Crear un registro (necesitamos envolver el diccionario en una lista)
                result = odoo.execute('account.move', 'create', [record])
                inserted_record_ids.append(result)

            insertion_history.append({
                'execution_id': execution_id,
                'new_records': inserted_record_ids
            })

            print(f"Se insertaron {len(inserted_record_ids)} nuevas facturas en account.move.")

        except Exception as e:
            print(f"Error durante la inserción: {str(e)}")

            # Intentar eliminar los registros ya insertados en caso de error
            if inserted_record_ids:
                try:
                    print(f"Intentando eliminar los registros insertados debido al error...")
                    odoo.execute('account.move', 'unlink', inserted_record_ids)
                    print("Registros eliminados correctamente.")
                except Exception as rollback_error:
                    print(f"Error al intentar eliminar registros: {str(rollback_error)}")

    else:
        print("No se encontraron registros válidos para insertar en account.move.")


In [252]:
# Prueba de la función de inserción
insert_invoices_with_rollback(partners_to_odoo_invoices, execution_id='5')


Se insertaron 2473 nuevas facturas en account.move.


In [56]:
#codigo para mostrar los ultimos registros en el modelo, tomando en cuenta la cantidad d eregistros que se insertaron en el Bulk insert anterior

#amount_untaxed amount_tax
i6 = odoo.search_read('account.move', [], ['id', 'partner_id', 'currency_id', 'amount_untaxed', 'amount_tax'], limit=None)

# Convertir los resultados a un DataFrame
inv_df = pd.DataFrame(i6)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_inv = inv_df.sort_values(by='id', ascending=False).head(2382)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_invoices = pd.DataFrame(last_inv)
last_invoices
#66064

,id,partner_id,currency_id,amount_untaxed,amount_tax
0,66064,"[34827, 16CXyqUDJkqSi1S5N]","[1, USD]",0.0,0.00
1,164,"[11, Gemini Furniture]","[1, USD]",0.0,0.00
3,163,False,"[1, USD]",0.0,0.00
8,162,"[10, Deco Addict]","[1, USD]",500.0,75.00
2,161,"[11, Gemini Furniture]","[1, USD]",25.0,3.75
4,160,"[11, Gemini Furniture]","[1, USD]",0.0,0.00
5,159,"[11, Gemini Furniture]","[1, USD]",0.0,0.00
6,158,"[11, Gemini Furniture]","[1, USD]",0.0,0.00
7,157,"[11, Gemini Furniture]","[1, USD]",0.0,0.00
10,156,"[11, Gemini Furniture]","[1, USD]",25.0,3.75


In [49]:
last_invoices.loc[last_invoices['amount_untaxed'] > 0]

,id,partner_id,currency_id,amount_untaxed,amount_tax
0,66062,"[108, Ayuntamiento De Bilbao, Departamento de ...","[1, USD]",940.0,141.0


In [327]:
#confirmar que no tengan impuestos ya que las facturas de agosto no tienen impuestos

#last_invoices.loc[last_invoices['amount_untaxed'] > 0]

,id,partner_id,currency_id,amount_untaxed,amount_tax,PartnerID,name


In [50]:
# Obtener los IDs de los últimos  registros de account.move y eliminarlos en base al lsitado de los ultimos ids o de un bloque de
#registros a eliminar
ids_to_delete = last_inv['id'].tolist()

# Eliminar los registros utilizando el método 'unlink' de Odoo
if ids_to_delete:
    odoo.execute('account.move', 'unlink', [ids_to_delete])

# Mostrar el número de registros eliminados
print(f"Se han eliminado {len(ids_to_delete)} registros de account.move.")

Se han eliminado 2382 registros de account.move.


In [388]:
#mostrar todos los campos de account.move

ii = odoo.search_read('account.move', [], [], limit=None)
ii_df = pd.DataFrame(ii)

In [389]:
ii_df

,id,campaign_id,source_id,medium_id,is_tax_computed_externally,is_avatax,sequence_prefix,sequence_number,activity_ids,activity_state,...,l10n_ar_afip_asoc_period_start,l10n_ar_afip_asoc_period_end,l10n_ar_boarding_permission_ids,timesheet_ids,timesheet_count,timesheet_encode_uom_id,timesheet_total_duration,document_type_internal_type,invoice_validation_type,l10n_ar_edi_warning
0,164,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
1,161,False,False,False,False,False,INV/2024/,11,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2,163,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
3,160,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
4,159,False,False,False,False,False,,0,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2298,19,False,False,False,False,False,BNK1/2024/,3,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2299,18,False,False,False,False,False,BNK1/2024/,2,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2300,17,False,False,False,False,False,BNK1/2024/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False
2301,11,False,False,False,False,False,RBILL/2023/05/,1,[],False,...,False,False,[],[],0,"[4, Hours]",0,False,False,False


In [255]:
ii_df['invoice_date']

NameError: name 'ii_df' is not defined

In [137]:
# Filtrar las columnas que contienen 'date' en su nombre
date_columns = [col for col in order.columns if 'tax' in col.lower()]

# Crear un nuevo dataframe solo con las columnas que contienen 'date'
date_df = order[date_columns]

# Mostrar el nuevo dataframe filtrado
date_df


,tax,tax_origin.country,taxes,line_item_taxes,tax_origin,is_taxed,tax_amount,tax_exempt_reason
0,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
1,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
2,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
3,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
4,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
...,...,...,...,...,...,...,...,...
2471,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
2465,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
2466,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured
2458,0,US,<class 'chargebee.models.invoice.Invoice.Tax'>,<class 'chargebee.models.invoice.Invoice.LineI...,<class 'chargebee.models.invoice.Invoice.TaxOr...,False,0,tax_not_configured


In [256]:
# Separar la columna move_id en dos columnas: move_id (numérico) y description para utilizar el partnerID para el
#Bulk insert rn account.move.line
last_invoices[['PartnerID', 'name']] = pd.DataFrame(last_invoices['partner_id'].tolist(), index=last_invoices.index)

# Convertir el campo move_id_num a tipo numérico
last_invoices['PartnerID'] = pd.to_numeric(last_invoices['PartnerID'], errors='coerce')

# Verificar el resultado
partner_invoices = last_invoices[['id', 'PartnerID', 'name']]
partner_invoices

,id,PartnerID,name
125,66061,35493,66d1e1c49112e41a78b90d4c
126,66060,35490,66d0a6efa9c6e736847a56e6
127,66059,35428,66cf6f0374ecf760b7a5b258
128,66058,35431,66cf6e434427b90002f38734
129,66057,35455,66ccdda3b1c5d119cf1a2412
...,...,...,...
120,63593,35586,540f3b9ee4b0492f27113fe7
121,63592,35589,53c42a63e4b08c5d83b878ff
122,63591,35590,536109b2e4b0ae650c483374
123,63590,35560,535680f7e4b03a7c45f66703


In [263]:
#renombro estos campos para que se pueda hacer un merge entre las invoices de Chargebee con estas que son de Odoo
renamed_partner_invoices = partner_invoices.rename(columns={'name': 'customer_id', 'id': 'invoice_id'})
renamed_partner_invoices

,invoice_id,PartnerID,customer_id
125,66061,35493,66d1e1c49112e41a78b90d4c
126,66060,35490,66d0a6efa9c6e736847a56e6
127,66059,35428,66cf6f0374ecf760b7a5b258
128,66058,35431,66cf6e434427b90002f38734
129,66057,35455,66ccdda3b1c5d119cf1a2412
...,...,...,...
120,63593,35586,540f3b9ee4b0492f27113fe7
121,63592,35589,53c42a63e4b08c5d83b878ff
122,63591,35590,536109b2e4b0ae650c483374
123,63590,35560,535680f7e4b03a7c45f66703


In [264]:
order['customer_id']

0       5acdf7d8e4b014cba3b1724b
1       5aa291c2e4b0c65fcc798ce8
2       5ad08c66e4b01bf0eefaf887
3       5cb4d5d0e4b0bd6bed7869fa
4       5b47de04e4b03c859495a0f7
                  ...           
2471    5a71ba83e4b07b95e31fc272
2465    5727680de4b06a6e4222f0c8
2466    53c42a63e4b08c5d83b878ff
2458    536109b2e4b0ae650c483374
2474    5a0c58c9e4b031a2c96d319c
Name: customer_id, Length: 2475, dtype: object

In [265]:
#unir los dataframes de Odoo y Chargebee para ver a que customer pertnece cada partner y que invoice_id de Odoo tiene ese partner
#last_invoices
lines_join = pd.merge(renamed_partner_invoices, order, on='customer_id', how='inner')
lines_join

,invoice_id,PartnerID,customer_id,id,po_number,subscription_id,recurring,status,price_type,date,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,66061,35493,66d1e1c49112e41a78b90d4c,187792,NaN,6okWDUMwqAQ63k1P,True,paid,tax_exclusive,2024-08-30 12:28:23-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66060,35490,66d0a6efa9c6e736847a56e6,187789,Cuenta test,16CXxzUMwehmC2IUg,True,paid,tax_exclusive,2024-08-30 11:38:38-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66059,35428,66cf6f0374ecf760b7a5b258,187717,Colombia,16CV2TUMqv4Dc2Ag8,True,paid,tax_exclusive,2024-08-29 12:06:00-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66058,35431,66cf6e434427b90002f38734,187720,Mexico,AzZnUfUMqw8ci2C2q,True,paid,tax_exclusive,2024-08-29 12:10:15-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,66057,35455,66ccdda3b1c5d119cf1a2412,187751,Argentina,16CV2TUMs0jeK371d,True,paid,tax_exclusive,2024-08-29 16:34:47-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5590,63593,35586,540f3b9ee4b0492f27113fe7,187909,Argentina - Transferencia Bancaria (Leadaki),AzyhzvUBeTc508oWU,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5591,63592,35589,53c42a63e4b08c5d83b878ff,187912,Argentina - Transferencia Bancaria (Leadaki),AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5592,63591,35590,536109b2e4b0ae650c483374,187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5593,63590,35560,535680f7e4b03a7c45f66703,187881,Argentina - Transferencia Bancaria (Leadaki),AzqB5PUBczDZj76jS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,NaN,line_item_discount,li_16BdL2UN1xX73CrTF,item_level_coupon,2400.0,CTASPRUEBA-100%-FOREVER,CTASPRUEBA-100%-FOREVER,NaN,NaN,NaN


In [267]:
lines_join_clean = lines_join.drop_duplicates('id')
lines_join_clean

,invoice_id,PartnerID,customer_id,id,po_number,subscription_id,recurring,status,price_type,date,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,66061,35493,66d1e1c49112e41a78b90d4c,187792,NaN,6okWDUMwqAQ63k1P,True,paid,tax_exclusive,2024-08-30 12:28:23-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66060,35490,66d0a6efa9c6e736847a56e6,187789,Cuenta test,16CXxzUMwehmC2IUg,True,paid,tax_exclusive,2024-08-30 11:38:38-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66059,35428,66cf6f0374ecf760b7a5b258,187717,Colombia,16CV2TUMqv4Dc2Ag8,True,paid,tax_exclusive,2024-08-29 12:06:00-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66058,35431,66cf6e434427b90002f38734,187720,Mexico,AzZnUfUMqw8ci2C2q,True,paid,tax_exclusive,2024-08-29 12:10:15-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,66057,35455,66ccdda3b1c5d119cf1a2412,187751,Argentina,16CV2TUMs0jeK371d,True,paid,tax_exclusive,2024-08-29 16:34:47-03:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5590,63593,35586,540f3b9ee4b0492f27113fe7,187909,Argentina - Transferencia Bancaria (Leadaki),AzyhzvUBeTc508oWU,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5591,63592,35589,53c42a63e4b08c5d83b878ff,187912,Argentina - Transferencia Bancaria (Leadaki),AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5592,63591,35590,536109b2e4b0ae650c483374,187913,Argentina - Transferencia Bancaria (Leadaki),16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5593,63590,35560,535680f7e4b03a7c45f66703,187881,Argentina - Transferencia Bancaria (Leadaki),AzqB5PUBczDZj76jS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,...,NaN,line_item_discount,li_16BdL2UN1xX73CrTF,item_level_coupon,2400.0,CTASPRUEBA-100%-FOREVER,CTASPRUEBA-100%-FOREVER,NaN,NaN,NaN


In [268]:
#esto demuestra que de todas las invoices generadas en Odoo 2 de ellas no tenian un tipo de moneda de USD
lines_join_clean['amount']

0       3900
1       1000
2          0
3       4000
4          0
        ... 
5590     901
5591       0
5592    3900
5593     381
5594     940
Name: amount, Length: 2473, dtype: int64

In [269]:
#aqui me encargo de renombrar campos de el registro de las ultimas invoices insertadas para poderlo unir con otro dataframe mas adelante
last_moves = last_invoices.rename(columns={'id': 'move_id_num'})
last_moves

,move_id_num,partner_id,currency_id,amount_untaxed,amount_tax,PartnerID,name
125,66061,"[35493, 66d1e1c49112e41a78b90d4c]","[1, USD]",0.0,0.0,35493,66d1e1c49112e41a78b90d4c
126,66060,"[35490, 66d0a6efa9c6e736847a56e6]","[1, USD]",0.0,0.0,35490,66d0a6efa9c6e736847a56e6
127,66059,"[35428, 66cf6f0374ecf760b7a5b258]","[1, USD]",0.0,0.0,35428,66cf6f0374ecf760b7a5b258
128,66058,"[35431, 66cf6e434427b90002f38734]","[1, USD]",0.0,0.0,35431,66cf6e434427b90002f38734
129,66057,"[35455, 66ccdda3b1c5d119cf1a2412]","[1, USD]",0.0,0.0,35455,66ccdda3b1c5d119cf1a2412
...,...,...,...,...,...,...,...
120,63593,"[35586, 540f3b9ee4b0492f27113fe7]","[1, USD]",0.0,0.0,35586,540f3b9ee4b0492f27113fe7
121,63592,"[35589, 53c42a63e4b08c5d83b878ff]","[1, USD]",0.0,0.0,35589,53c42a63e4b08c5d83b878ff
122,63591,"[35590, 536109b2e4b0ae650c483374]","[1, USD]",0.0,0.0,35590,536109b2e4b0ae650c483374
123,63590,"[35560, 535680f7e4b03a7c45f66703]","[1, USD]",0.0,0.0,35560,535680f7e4b03a7c45f66703


In [86]:
#una busqueda de registro por id


#last_moves.loc[last_moves['move_id_num'] == 20097]

,move_id_num,partner_id,currency_id,PartnerID,name


In [270]:
print_column_types(last_invoices)

id 	 int64
partner_id 	 object
currency_id 	 object
amount_untaxed 	 float64
amount_tax 	 float64
PartnerID 	 int64
name 	 object


In [271]:
order

,id,po_number,customer_id,subscription_id,recurring,status,price_type,date,due_date,net_term_days,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_id_notes_expanded
0,185392,Argentina - Transferencia Bancaria,5acdf7d8e4b014cba3b1724b,169uzAU4TrwKK33PY,True,paid,tax_exclusive,2024-08-01 05:08:00-03:00,2024-08-01 05:08:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,185393,Argentina - Transferencia Bancaria,5aa291c2e4b0c65fcc798ce8,6or0jU5PxxMNEhuk,True,paid,tax_exclusive,2024-08-01 07:17:00-03:00,2024-08-01 07:17:00-03:00,0,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,185394,Argentina - Transferencia Bancaria,5ad08c66e4b01bf0eefaf887,AzZN3tT4OBaWmiMY,True,paid,tax_exclusive,2024-08-01 07:39:00-03:00,2024-08-01 07:39:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,185395,Argentina - Transferencia Bancaria,5cb4d5d0e4b0bd6bed7869fa,16CY2RUEdBvEE1YCI,True,paid,tax_exclusive,2024-08-01 07:54:00-03:00,2024-08-01 07:54:00-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,185396,Chile,5b47de04e4b03c859495a0f7,Izy9TIBR04obyPV06,True,paid,tax_exclusive,2024-08-01 08:05:08-03:00,2024-08-01 08:05:08-03:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,187910,Argentina - Transferencia Bancaria (Leadaki),5a71ba83e4b07b95e31fc272,AzqBxKUBeW38u8EGG,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2465,187911,Argentina - Transferencia Bancaria (Leadaki),5727680de4b06a6e4222f0c8,AzyhzvUBeSCea8nZZ,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2466,187912,Argentina - Transferencia Bancaria (Leadaki),53c42a63e4b08c5d83b878ff,AzqBxKUBeN2WG88Ao,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2458,187913,Argentina - Transferencia Bancaria (Leadaki),536109b2e4b0ae650c483374,16CY4nUBd1nGm7OLS,True,paid,tax_exclusive,2024-09-01 00:00:00-03:00,2024-09-01 00:00:00-03:00,0,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
#a partir del dataframe de Chargebee, me toca aislar todos los campos que incluyen los line items, descuentos y notas de credito

Chargebee_li = order[['id_line_items_expanded', 'date_from', 'date_to', 'unit_amount', 'quantity', 'amount', 'pricing_model', 'is_taxed',
       'tax_amount', 'object_line_items_expanded', 'description', 'entity_type', 'entity_id', 'entity_description', 'tax_exempt_reason'
       , 'discount_amount', 'item_level_discount_amount', 'starting_unit', 'ending_unit', 'quantity_used',
       'unit_amount_line_item_tiers_expanded', 'line_item_id', 'txn_id', 'applied_amount', 'applied_at', 'txn_status', 'txn_date',
       'txn_amount', 'applied_amount_applied_credits_expanded', 'applied_at_applied_credits_expanded', 'cn_id', 'cn_reason_code',
       'cn_create_reason_code', 'cn_date', 'cn_status', 'cn_id_adjustment_credit_notes_expanded',
       'cn_reason_code_adjustment_credit_notes_expanded', 'cn_create_reason_code_adjustment_credit_notes_expanded',
       'cn_date_adjustment_credit_notes_expanded', 'cn_total', 'cn_status_adjustment_credit_notes_expanded',
       'cn_id_issued_credit_notes_expanded', 'cn_reason_code_issued_credit_notes_expanded',
       'cn_create_reason_code_issued_credit_notes_expanded', 'cn_date_issued_credit_notes_expanded',
       'cn_total_issued_credit_notes_expanded', 'cn_status_issued_credit_notes_expanded', 'created_at', 'attempt', 'dunning_type',
       'transaction_id', 'txn_status_dunning_attempts_expanded', 'txn_amount_dunning_attempts_expanded', 'object_discounts_expanded',
       'entity_type_discounts_expanded', 'description_discounts_expanded', 'amount_discounts_expanded', 'discount_type',
       'discount_percentage', 'object_line_item_discounts_expanded', 'line_item_id_line_item_discounts_expanded',
       'discount_type_line_item_discounts_expanded', 'discount_amount_line_item_discounts_expanded', 'coupon_id',
       'entity_id_line_item_discounts_expanded', 'note', 'entity_type_notes_expanded', 'entity_type_notes_expanded']]

In [273]:
cbli = Chargebee_li.drop_duplicates('id_line_items_expanded')
cbli

,id_line_items_expanded,date_from,date_to,unit_amount,quantity,amount,pricing_model,is_taxed,tax_amount,object_line_items_expanded,...,discount_percentage,object_line_item_discounts_expanded,line_item_id_line_item_discounts_expanded,discount_type_line_item_discounts_expanded,discount_amount_line_item_discounts_expanded,coupon_id,entity_id_line_item_discounts_expanded,note,entity_type_notes_expanded,entity_type_notes_expanded
0,li_6olL2UKBV7cg22wd,2024-08-01,2024-09-01 05:08:00-03:00,8900,1,8900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,li_6olL2UKBV7cm22we,2024-08-01,2024-09-01 05:08:00-03:00,0,1,0,tiered,False,0,line_item,...,NaN,line_item_discount,li_Azz6aoUKC1bN81M2q,item_level_coupon,2000.0,Migra2024Premium2Users,Migra2024Premium2Users,NaN,NaN,NaN
2,li_Azz6aoUKC1bN11M2p,2024-08-01,2024-09-01 07:17:00-03:00,8900,1,8900,flat_fee,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,li_Azz6aoUKC1bN81M2q,2024-08-01,2024-09-01 07:17:00-03:00,750,4,3000,tiered,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,li_AzyjxKUKC78gO1LOQ,2024-08-01,2024-09-01 07:39:00-03:00,39,100,3900,stairstep,False,0,line_item,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,li_16BdPRULMerQU7LqT,2024-08-13,2024-09-13 17:24:33-03:00,12900,1,12900,flat_fee,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2465,li_AzZnNZULMe4v13Smf,2024-08-13,2024-09-13 17:21:28-03:00,12900,1,12900,flat_fee,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2466,li_AzZnNZULMe4v53Smh,2024-08-13,2024-09-13 17:21:28-03:00,0,1,0,flat_fee,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2458,li_Azyk7rULMR9zY1kFo,2024-08-13,2024-09-13 16:30:08-03:00,3900,1,3900,flat_fee,False,0,line_item,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [274]:
# Obtener todos los campos del modelo account.move.line
fields_move_line = odoo.execute('account.move.line', 'fields_get', {})

# Filtrar los campos que contengan 'tax' o 'tx' y extraer solo los nombres de los campos
filtered_fields = [field for field in fields_move_line.keys() if 'tax' in field or 'tx' in field]

# Mostrar los campos filtrados
taxes_df = pd.DataFrame(filtered_fields)
taxes_df

,0
0,tax_ids
1,group_tax_id
2,tax_line_id
3,tax_group_id
4,tax_base_amount
5,tax_repartition_line_id
6,tax_tag_ids
7,tax_tag_invert
8,tax_calculation_rounding_method
9,tax_key


In [70]:
#tax_ids
#group_tax_id
#tax_line_id
#tax_group_id
#tax_base_amount
#tax_repartition_line_id
#tax_tag_ids
#tax_tag_invert
#tax_calculation_rounding_method
#tax_key
#compute_all_tax
#compute_all_tax_dirty
#non_deductible_tax_value
#tax_settlement_move_id
#tax_state

In [277]:
#codigo encargado de geenrar el Bulk insert en account.move.line en base al dataframe que muestra a todos los invoices de account.move
#que esta unido a los de chargebee para que tengan tambien los campos de invoice de Chargebee y asi poder insertar tambien los campos
#que hagan falta migrar de Chargebee a Odoo
import signal

# Variable global para indicar si hubo una interrupción manual o del sistema
interrupted = False

# Función para manejar señales de interrupción (como CTRL+C o kill)
def handle_interrupt(signum, frame):
    global interrupted
    print(f"Señal {signum} recibida. Iniciando rollback...")
    interrupted = True  # Marcar interrupción
    raise Exception("Proceso interrumpido manualmente o por el sistema.")

# Conectar las señales de interrupción al manejador de interrupciones
signal.signal(signal.SIGINT, handle_interrupt)   # CTRL+C
signal.signal(signal.SIGTERM, handle_interrupt)  # kill command

def insert_account_move_lines_with_rollback(lines_join_clean, execution_id):
    global interrupted
    records_to_insert = []
    inserted_record_ids = []
    unique_combinations = set()  # Para almacenar combinaciones únicas de move_id y partner_id

    # Eliminar duplicados en el DataFrame antes de procesar
    lines_join_clean = lines_join_clean.drop_duplicates(subset=['id', 'PartnerID'])

    # Iterar sobre las filas del DataFrame sin duplicar
    for _, row in lines_join_clean.iterrows():
        move_id = row['invoice_id']
        partner_id = row['PartnerID']
        name = row['description']
        quantity = row['quantity']
        amount = row['amount']
        tax = row['tax']

        # Evitar duplicados verificando la combinación única de move_id y partner_id
        if (move_id, partner_id) in unique_combinations:
            print(f"Combinación duplicada detectada: move_id {move_id} y partner_id {partner_id}. Saltando.")
            continue  # Saltar duplicados

        unique_combinations.add((move_id, partner_id))  # Añadir la combinación a la lista de únicas

        record = {
            'move_id': move_id,
            'name': name,  # Descripción de la línea
            'quantity': quantity,  # Cantidad
            'price_unit': amount/100,  # Precio unitario
            'debit': 0.0,  # Débito
            'partner_id': partner_id,  # ID del cliente
            'currency_id': 1 , # ID de la moneda
            'tax_ids': False,
            'group_tax_id': False,
            'tax_line_id': False,
            'tax_group_id': False,
            'tax_base_amount': tax,
            'tax_repartition_line_id': False,
            'tax_tag_ids': False,
            'tax_tag_invert': False,
            'tax_calculation_rounding_method': False,
            'compute_all_tax_dirty': True,
            'tax_state': False
            


        }

        # Verificar si 'move_id' es válido
        if 'move_id' not in record or not record['move_id']:
            print(f"Registro inválido: {record}")
            continue

        records_to_insert.append(record)

    if records_to_insert:
        try:
            # Realizar inserciones en un bucle
            for record in records_to_insert:
                if interrupted:
                    raise Exception("Interrupción detectada antes de insertar.")
                
                # Crear un registro (necesitamos envolver el diccionario en una lista)
                result = odoo.execute('account.move.line', 'create', [record])
                inserted_record_ids.append(result)

            print(f"Se insertaron {len(inserted_record_ids)} nuevas líneas en account.move.line.")

        except Exception as e:
            print(f"Error durante la inserción: {str(e)}")

            # Intentar eliminar los registros ya insertados en caso de error
            if inserted_record_ids:
                try:
                    print(f"Intentando eliminar los registros insertados debido al error...")
                    odoo.execute('account.move.line', 'unlink', inserted_record_ids)
                    print("Registros eliminados correctamente.")
                except Exception as rollback_error:
                    print(f"Error al intentar eliminar registros: {str(rollback_error)}")

    else:
        print("No se encontraron registros válidos para insertar en account.move.line.")

In [278]:
# Llamar la función con el DataFrame partner_invoices
insert_account_move_lines_with_rollback(lines_join_clean, execution_id='2')

Combinación duplicada detectada: move_id 66055 y partner_id 34967. Saltando.
Combinación duplicada detectada: move_id 66042 y partner_id 34405. Saltando.
Combinación duplicada detectada: move_id 66040 y partner_id 34394. Saltando.
Combinación duplicada detectada: move_id 66026 y partner_id 33474. Saltando.
Combinación duplicada detectada: move_id 66021 y partner_id 33465. Saltando.
Combinación duplicada detectada: move_id 65999 y partner_id 33992. Saltando.
Combinación duplicada detectada: move_id 65988 y partner_id 33559. Saltando.
Combinación duplicada detectada: move_id 65988 y partner_id 33559. Saltando.
Combinación duplicada detectada: move_id 65984 y partner_id 35341. Saltando.
Combinación duplicada detectada: move_id 65984 y partner_id 35341. Saltando.
Combinación duplicada detectada: move_id 65984 y partner_id 35341. Saltando.
Combinación duplicada detectada: move_id 65984 y partner_id 35341. Saltando.
Combinación duplicada detectada: move_id 65984 y partner_id 35341. Saltando.

KeyboardInterrupt: 

In [287]:
#tax_ids
#group_tax_id
#tax_line_id
#tax_group_id
#tax_base_amount
#tax_repartition_line_id
#tax_tag_ids
#tax_tag_invert
#tax_calculation_rounding_method
#tax_key
#compute_all_tax
#compute_all_tax_dirty
#non_deductible_tax_value
#tax_settlement_move_id
#tax_state
i7 = odoo.search_read('account.move.line', [], ['id', 'move_id', 'tax_base_amount', 
'compute_all_tax_dirty', 'non_deductible_tax_value', 'tax_settlement_move_id', 'tax_state'], limit=None)

# Convertir los resultados a un DataFrame
df_line_items = pd.DataFrame(i7)

# Ordenar el DataFrame por la columna 'id' en orden descendente y seleccionar la cantidad de los ultimos registros que hay que ver
last_li_inv = df_line_items.sort_values(by='id', ascending=False).head(184)

# Mostrar el DataFrame con los últimos registros mostrando 'id' y 'name'
last_invoices_lines = pd.DataFrame(last_li_inv)
last_invoices_lines

,id,move_id,tax_base_amount,compute_all_tax_dirty,non_deductible_tax_value,tax_settlement_move_id,tax_state
4,480,"[163, Draft Invoice (* 163)]",0.0,True,0.0,False,False
3,479,"[163, Draft Invoice (* 163)]",0.0,True,0.0,False,False
7,478,"[162, INV/2024/00012 (123456)]",0.0,True,0.0,False,False
6,477,"[162, INV/2024/00012 (123456)]",500.0,False,0.0,False,to_settle
5,476,"[162, INV/2024/00012 (123456)]",0.0,True,0.0,False,False
...,...,...,...,...,...,...,...
54,5,"[1, INV/2024/00002]",0.0,True,0.0,False,False
63,4,"[16, MISC/2024/07/0002]",0.0,True,0.0,False,False
62,3,"[16, MISC/2024/07/0002]",0.0,True,0.0,False,False
65,2,"[15, MISC/2024/07/0001]",0.0,True,0.0,False,False


In [282]:
last_invoices_lines.loc[last_invoices_lines['tax_settlement_move_id'] != False]

,id,move_id,tax_base_amount,compute_all_tax_dirty,non_deductible_tax_value,tax_settlement_move_id,tax_state


In [283]:
# Separar la columna move_id en dos columnas: move_id (numérico) y description
last_invoices_lines[['move_id_num', 'description']] = pd.DataFrame(last_invoices_lines['move_id'].tolist(), index=last_invoices_lines.index)

# Convertir el campo move_id_num a tipo numérico
last_invoices_lines['move_id_num'] = pd.to_numeric(last_invoices_lines['move_id_num'], errors='coerce')

# Verificar el resultado
last_invoices_lines_refined = last_invoices_lines[['id', 'move_id_num', 'description']]
last_invoices_lines_refined

,id,move_id_num,description
95,68585,65954,Draft Invoice (* 65954)
94,68584,65954,Draft Invoice (* 65954)
195,68583,65955,Draft Invoice (* 65955)
194,68582,65955,Draft Invoice (* 65955)
115,68581,65956,Draft Invoice (* 65956)
...,...,...,...
60,68406,66059,Draft Invoice (* 66059)
59,68405,66060,Draft Invoice (* 66060)
58,68404,66060,Draft Invoice (* 66060)
57,68403,66061,Draft Invoice (* 66061)


In [284]:
#last_moves que genere antes ahora lo uno con el dataframe que me permite separar el move_id de su numero y descripcion en el modelo
#account.move.line porque ese id lo muestra con un array tipo asi [45268, 'Draft Invoice (* 45268)']
lines_i_join = pd.merge(last_moves, last_invoices_lines_refined, on='move_id_num', how='inner')
lines_i_join

,move_id_num,partner_id,currency_id,amount_untaxed,amount_tax,PartnerID,name,id,description
0,66061,"[35493, 66d1e1c49112e41a78b90d4c]","[1, USD]",0.0,0.0,35493,66d1e1c49112e41a78b90d4c,68403,Draft Invoice (* 66061)
1,66061,"[35493, 66d1e1c49112e41a78b90d4c]","[1, USD]",0.0,0.0,35493,66d1e1c49112e41a78b90d4c,68402,Draft Invoice (* 66061)
2,66060,"[35490, 66d0a6efa9c6e736847a56e6]","[1, USD]",0.0,0.0,35490,66d0a6efa9c6e736847a56e6,68405,Draft Invoice (* 66060)
3,66060,"[35490, 66d0a6efa9c6e736847a56e6]","[1, USD]",0.0,0.0,35490,66d0a6efa9c6e736847a56e6,68404,Draft Invoice (* 66060)
4,66059,"[35428, 66cf6f0374ecf760b7a5b258]","[1, USD]",0.0,0.0,35428,66cf6f0374ecf760b7a5b258,68407,Draft Invoice (* 66059)
...,...,...,...,...,...,...,...,...,...
179,65956,"[35132, 6679c5e1a21e4473c7109488]","[1, USD]",0.0,0.0,35132,6679c5e1a21e4473c7109488,68580,Draft Invoice (* 65956)
180,65955,"[34125, 6675b638540f0c743278057a]","[1, USD]",0.0,0.0,34125,6675b638540f0c743278057a,68583,Draft Invoice (* 65955)
181,65955,"[34125, 6675b638540f0c743278057a]","[1, USD]",0.0,0.0,34125,6675b638540f0c743278057a,68582,Draft Invoice (* 65955)
182,65954,"[35308, 667500aeabac737a817179b8]","[1, USD]",0.0,0.0,35308,667500aeabac737a817179b8,68585,Draft Invoice (* 65954)


In [415]:
#buscar un registro en account.move.line por medio de su id
lines_i_join.loc[lines_i_join['move_id_num']==20152]

,move_id_num,partner_id,currency_id,amount_untaxed,amount_tax,PartnerID,name,id,description


In [285]:
# Reversar los movimientos contables
move_ids = last_invoices_lines['move_id_num'].dropna().astype(int).tolist()  # Convertir a enteros y eliminar valores NaN
print("IDs de movimientos a procesar:", move_ids)


IDs de movimientos a procesar: [65954, 65954, 65955, 65955, 65956, 65956, 65957, 65957, 65958, 65958, 65959, 65959, 65960, 65960, 65961, 65961, 65962, 65962, 65963, 65963, 65964, 65964, 65965, 65965, 65966, 65966, 65967, 65967, 65968, 65968, 65969, 65969, 65970, 65970, 65971, 65971, 65973, 65973, 65976, 65976, 65977, 65977, 65978, 65978, 65984, 65984, 65985, 65985, 65988, 65988, 65989, 65989, 65990, 65990, 65991, 65991, 65992, 65992, 65993, 65993, 65994, 65994, 65995, 65995, 65996, 65996, 65997, 65997, 65999, 65999, 66000, 66000, 66001, 66001, 66002, 66002, 66003, 66003, 66004, 66004, 66005, 66005, 66006, 66006, 66007, 66007, 66008, 66008, 66009, 66009, 66010, 66010, 66011, 66011, 66012, 66012, 66013, 66013, 66014, 66014, 66015, 66015, 66016, 66016, 66017, 66017, 66018, 66018, 66019, 66019, 66021, 66021, 66022, 66022, 66023, 66023, 66024, 66024, 66026, 66026, 66027, 66027, 66028, 66028, 66029, 66029, 66030, 66030, 66031, 66031, 66032, 66032, 66033, 66033, 66034, 66034, 66035, 66035, 66

In [286]:
# Reversar los movimientos contables osea eliminar todos los registros contables, lo que consiste en eliminar los line items de cada
# invoice y a la vez tambien se elimine el registro padre que es el de account.move, pero esta manera de liminarlos fue la que pude
#utilizar :(
move_ids = last_invoices_lines_refined['move_id_num'].dropna().astype(int).tolist()  # Convertir a enteros y eliminar valores NaN

# Usar un conjunto para asegurarse de no procesar IDs duplicados
processed_ids = set()

if move_ids:
    try:
        for move_id in move_ids:
            # Verificar si el ID ya ha sido procesado
            if move_id in processed_ids:
                print(f"ID de movimiento {move_id} ya fue procesado, saltando...")
                continue
            
            print(f"Procesando ID de movimiento: {move_id} (Tipo: {type(move_id)})")
            
            # Verificar si el movimiento existe
            move_record = odoo.execute('account.move', 'search_read', [[('id', '=', move_id)]], {'fields': ['state']})
            
            if move_record:
                if move_record[0]['state'] != 'draft':
                    # Si el estado no es 'draft', cambiarlo a 'draft'
                    odoo.execute('account.move', 'button_draft', [[move_id]])
                    print(f"Asiento contable {move_id} puesto en estado borrador.")
                
                # Eliminar el movimiento contable
                odoo.execute('account.move', 'unlink', [[move_id]])
                print(f"Asiento contable {move_id} eliminado.")
                
                # Marcar el ID como procesado
                processed_ids.add(move_id)
            else:
                # Si no encuentra el movimiento, continuar con el siguiente
                print(f"Asiento contable {move_id} no encontrado, posiblemente ya ha sido eliminado previamente.")
                processed_ids.add(move_id)  # Marcarlo como procesado para evitar reintentos
                continue  # Sigue con el siguiente movimiento

        print(f"Se han revertido y eliminado {len(processed_ids)} asientos contables.")
        
    except Exception as e:
        print(f"Error al revertir/eliminar los asientos: {str(e)}")
else:
    print("No se encontraron IDs válidos para revertir/eliminar.")


Procesando ID de movimiento: 65954 (Tipo: <class 'int'>)
Asiento contable 65954 eliminado.
ID de movimiento 65954 ya fue procesado, saltando...
Procesando ID de movimiento: 65955 (Tipo: <class 'int'>)
Asiento contable 65955 eliminado.
ID de movimiento 65955 ya fue procesado, saltando...
Procesando ID de movimiento: 65956 (Tipo: <class 'int'>)
Asiento contable 65956 eliminado.
ID de movimiento 65956 ya fue procesado, saltando...
Procesando ID de movimiento: 65957 (Tipo: <class 'int'>)
Asiento contable 65957 eliminado.
ID de movimiento 65957 ya fue procesado, saltando...
Procesando ID de movimiento: 65958 (Tipo: <class 'int'>)
Asiento contable 65958 eliminado.
ID de movimiento 65958 ya fue procesado, saltando...
Procesando ID de movimiento: 65959 (Tipo: <class 'int'>)
Asiento contable 65959 eliminado.
ID de movimiento 65959 ya fue procesado, saltando...
Procesando ID de movimiento: 65960 (Tipo: <class 'int'>)
Asiento contable 65960 eliminado.
ID de movimiento 65960 ya fue procesado, salt

In [ ]:
#habian 94 registros

In [77]:
# Simulación de los impuestos disponibles en Odoo (ejemplo)
TAXES = {
    'iva15_llc': 1,  # ID del impuesto iva 15%
    'iva21_arg': 2,  # ID del impuesto iva 21% (por ejemplo)
}

def create_move(odoo):
    """
    Esta función simula la creación de un movimiento contable (account.move).
    """
    move_vals = {
        'journal_id': 1,  # ID del diario contable
        'date': '2024-10-10',  # Fecha del movimiento
        'ref': 'Referencia de prueba',  # Referencia del movimiento
    }
    move_id = odoo.create('account.move', [move_vals])  # Crear un nuevo account.move
    return move_id

def get_aml_vals(product_id, partner_id, price, quantity, tax_name):
    """
    Función que retorna los valores de una línea de 'account.move.line'.
    """
    return {
        'product_id': product_id,  # ID del producto
        'partner_id': partner_id,  # ID del cliente
        'name': 'Venta de producto',  # Descripción
        'quantity': quantity,  # Cantidad
        'price_unit': price,  # Precio unitario
        'debit': 0.0,  # Débito (en caso de ser necesario)
        'credit': price * quantity,  # Crédito
        'currency_id': 1,  # ID de la moneda
        'tax_ids': [(6, 0, [TAXES[tax_name]])],  # Impuesto aplicado
    }

def insert_lines_in_batch(odoo, products, partner_id):
    """
    Función que inserta líneas de account.move.line en batch para un account.move.
    
    - products: Lista de productos con su información.
    - partner_id: ID del cliente/partner para asociar a las líneas.
    """
    # 1. Crear el movimiento contable (account.move)
    move_id = create_move(odoo)

    # 2. Preparar las líneas de account.move.line
    aml_vals = []
    
    for product in products:
        product_id = product['product_id']
        price = product['price']
        quantity = product['quantity']
        tax_name = product['tax_name']  # Nombre del impuesto
        
        # Generar los valores de la línea
        aml_vals.append(get_aml_vals(product_id, partner_id, price, quantity, tax_name))
    
    # 3. Insertar las líneas de account.move.line usando create
    try:
        for aml in aml_vals:
            aml['move_id'] = move_id  # Asignar el ID del movimiento contable
            odoo.create('account.move.line', [aml])  # Crear las líneas individualmente
        print(f"Líneas insertadas correctamente en el movimiento con ID {move_id}.")
    except Exception as e:
        print("Error al insertar líneas:", str(e))

# Ejemplo de uso:
products = [
    {'product_id': 101, 'price': 100.0, 'quantity': 1, 'tax_name': 'iva15_llc'},  # Producto 1
    {'product_id': 102, 'price': 200.0, 'quantity': 2, 'tax_name': 'iva21_arg'},  # Producto 2
]

partner_id = 34827  # ID del cliente

# Insertar en batch
insert_lines_in_batch(odoo, products, partner_id)


Líneas insertadas correctamente en el movimiento con ID 66074.


In [84]:
def insert_lines_in_batch(odoo, products, partner_id):
    """
    Función que inserta líneas de account.move.line en batch para un account.move.
    
    - products: Lista de productos con su información.
    - partner_id: ID del cliente/partner para asociar a las líneas.
    """
    # 1. Crear el movimiento contable (account.move)
    move_id = create_move(odoo)

    # 2. Preparar las líneas de account.move.line
    aml_vals = []
    
    total_debit = 0  # Acumular total de débitos
    for product in products:
        product_id = product['product_id']
        price = product['price']
        quantity = product['quantity']
        
        # Generar los valores de la línea sin impuestos
        aml_vals.append(get_aml_vals(product_id, partner_id, price, quantity))
        total_debit += price * quantity  # Sumar al total de débitos

    # Agregar una línea de crédito para equilibrar
    aml_vals.append({
        'product_id': False,  # Sin producto
        'partner_id': partner_id,  # ID del cliente
        'name': 'Contrapartida',  # Descripción
        'quantity': 1,  # Cantidad de 1 para el crédito
        'price_unit': total_debit,  # Usar el total de débitos como precio unitario
        'debit': 0.0,  # Sin débito
        'credit': total_debit,  # Total de créditos
        'currency_id': 1,  # ID de la moneda
    })

    # Verificar si los totales de débitos y créditos están balanceados
    total_credit = sum(aml['credit'] for aml in aml_vals)
    if total_debit != total_credit:
        print("Error: Los totales de débitos y créditos no están balanceados.")
        return

    # 3. Insertar las líneas de account.move.line usando create
    try:
        for aml in aml_vals:
            aml['move_id'] = move_id  # Asignar el ID del movimiento contable
            odoo.create('account.move.line', [aml])  # Crear las líneas individualmente
        print(f"Líneas insertadas correctamente en el movimiento con ID {move_id}.")
    except Exception as e:
        print("Error al insertar líneas:", str(e))

# Ejemplo de uso:
products = [
    {'product_id': 101, 'price': 100.0, 'quantity': 1},  # Producto 1
    {'product_id': 102, 'price': 200.0, 'quantity': 2},  # Producto 2
]

partner_id = 34827  # ID del cliente

# Insertar en batch
insert_lines_in_batch(odoo, products, partner_id)


{'jsonrpc': '2.0', 'id': 54, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.UserError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in route_

In [80]:
#hacer el llamado a acoount.move.line para mas adelante poder
# Especificar los campos que deseas leer
fields_to_read = ['id', 'move_id', 'product_id', 'quantity', 'price_unit', 'debit', 'credit', 'partner_id',
                  ]

# Realizar el llamado al modelo account.move.line con un límite de registros
line_items = odoo.search_read('account.move.line', [], fields_to_read, limit=4530)

# Convertir los resultados a un DataFrame
li_df = pd.DataFrame(line_items)
li_df
#68593

,id,move_id,product_id,quantity,price_unit,debit,credit,partner_id
0,68593,"[66064, Draft Invoice (* 66064)]",False,1.0,28.75,0.00,28.75,"[34827, 16CXyqUDJkqSi1S5N]"
1,68594,"[66064, Draft Invoice (* 66064)]",False,0.0,0.00,28.75,0.00,"[34827, 16CXyqUDJkqSi1S5N]"
2,68595,"[66068, Draft Entry (* 66068) (Referencia de p...","[101, Repair Services]",1.0,100.00,0.00,100.00,"[34827, 16CXyqUDJkqSi1S5N]"
3,68596,"[66068, Draft Entry (* 66068) (Referencia de p...",False,0.0,0.00,0.00,60.00,"[34827, 16CXyqUDJkqSi1S5N]"
4,68597,"[66068, Draft Entry (* 66068) (Referencia de p...",False,1.0,0.00,575.00,0.00,False
...,...,...,...,...,...,...,...,...
101,36,"[11, RBILL/2023/05/0001]",False,0.0,0.00,0.00,81.17,"[14, Azure Interior]"
102,49,"[11, RBILL/2023/05/0001]",False,0.0,0.00,622.27,0.00,"[14, Azure Interior]"
103,15,"[7, BILL/2023/05/0001 (INV/2023/0057)]",False,1.0,541.10,541.10,0.00,"[14, Azure Interior]"
104,32,"[7, BILL/2023/05/0001 (INV/2023/0057)]",False,0.0,0.00,81.17,0.00,"[14, Azure Interior]"


In [113]:
line_id = empties = li_df.loc[li_df['id'] == 28629]
line_id

,id,move_id,product_id,quantity,price_unit,debit,credit,partner_id


In [295]:
# Obtener la descripción del modelo 'account.move.line'
fields_description = odoo.execute('account.move.line', 'fields_get', [], {'attributes': ['string', 'type']})
lines = pd.DataFrame(fields_description)
lines

,analytic_distribution,analytic_distribution_search,analytic_precision,move_id,journal_id,company_id,company_currency_id,move_name,parent_state,date,...,invoice_origin,payment_matched_amount,l10n_pe_group_id,filter_amount,tax_settlement_move_id,tax_state,subscription_id,subscription_mrr,move_line_ids,withholding_id
string,Analytic Distribution,Analytic Distribution Search,Analytic Precision,Journal Entry,Journal,Company,Company Currency,Number,Status,Date,...,Origin,Payment Matched Amount,Group,Filter Amount,Tax Settlement Move,Tax State,Subscription,Monthly Recurring Revenue,Related Stock Moves,Withholding
type,json,json,integer,many2one,many2one,many2one,many2one,char,selection,date,...,char,monetary,many2one,float,many2one,selection,many2one,monetary,many2many,many2one
searchable,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Mostrar las columnas (campos) del modelo
for field, description in fields_description.items():
    print(f"Campo: {field}, Tipo: {description['type']}, Descripción: {description['string']}")

Campo: analytic_distribution, Tipo: json, Descripción: Analytic Distribution
Campo: analytic_distribution_search, Tipo: json, Descripción: Analytic Distribution Search
Campo: analytic_precision, Tipo: integer, Descripción: Analytic Precision
Campo: move_id, Tipo: many2one, Descripción: Journal Entry
Campo: journal_id, Tipo: many2one, Descripción: Journal
Campo: company_id, Tipo: many2one, Descripción: Company
Campo: company_currency_id, Tipo: many2one, Descripción: Company Currency
Campo: move_name, Tipo: char, Descripción: Number
Campo: parent_state, Tipo: selection, Descripción: Status
Campo: date, Tipo: date, Descripción: Date
Campo: invoice_date, Tipo: date, Descripción: Invoice/Bill Date
Campo: ref, Tipo: char, Descripción: Reference
Campo: is_storno, Tipo: boolean, Descripción: Company Storno Accounting
Campo: sequence, Tipo: integer, Descripción: Sequence
Campo: move_type, Tipo: selection, Descripción: Type
Campo: account_id, Tipo: many2one, Descripción: Account
Campo: name, Tip

In [57]:
odoo.create('account.move.line', [{
    'move_id': 66064,  # ID del movimiento contable (relacionado con account.move)
    #'account_id': 3,  # ID de la cuenta contable
    'name': 'Venta de producto',  # Descripción de la línea
    'quantity': 1.0,  # Cantidad
    'price_unit': 28.75,  # Precio unitario
    'debit': 0.0,  # Débito
    #'credit': 28.75,  # Crédito
    'partner_id': 34827,  # ID del cliente
    #'company_id': 1,  # ID de la compañía
    'currency_id': 1,  # ID de la moneda
    #'date_maturity': '2024-09-30',  # Fecha de vencimiento
    #'analytic_account_id': False,  # ID de la cuenta analítica (si aplica)
    #'tax_ids': [(6, 0, [])],  # Impuestos aplicables (lista vacía si no hay)
}])


68593

In [ ]:
#66064   34827

In [151]:
#ahora voy a eliminar esos registros ya que aun no habia corregido los customers ya que hay duplicados
move_to_delete = 28623  # Reemplazar con el ID real que hay que eliminar

# Realizar la operación de eliminación
odoo.execute('account.move.line', 'unlink', [move_to_delete])

{'jsonrpc': '2.0', 'id': 28, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.ValidationError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in 

Exception: Command has failed

In [136]:
# Búsqueda de productos en el modelo 'product.product'
product_search = odoo.search_read('product.product', [], ['id', 'name'], limit=None)

# Convertir los resultados a un DataFrame para una mejor visualización
products_df = pd.DataFrame(product_search)

# Mostrar los productos encontrados
products_df


,id,name
0,94,Ad-Valorem
1,88,Tariff
2,86,Import Clearance
3,89,Guard Service
4,87,Statistics Rate
...,...,...
104,112,Senior Architect (Invoice on Timesheets)
105,110,Service on Timesheets
106,37,Top-up eWallet
107,4,Virtual Home Staging


In [153]:
products_df.loc[products_df['id'] == 118]

,id,name
73,118,Producto de ejemplo


In [70]:
# Datos del nuevo producto a insertar
new_product_data = {
    'name': 'Producto de ejemplo',
    'type': 'consu',  # tipo de producto ('consu' = Consumible, 'service' = Servicio, 'product' = Almacenable)
    'list_price': 100.00,  # Precio de venta
    'standard_price': 70.00,  # Precio de coste
    'default_code': 'PROD_EX',  # Código interno del producto
    'sale_ok': True,  # Habilitar para ventas
    'purchase_ok': True  # Habilitar para compras
}

# Insertar el nuevo producto usando el método Odoo.create
try:
    new_product_id = odoo.create('product.product', [new_product_data])
    print(f"Producto insertado con éxito. ID: {new_product_id}")
except Exception as e:
    print(f"Error durante la inserción del producto: {str(e)}")


Producto insertado con éxito. ID: 118


In [154]:
#ahora voy a eliminar esos registros ya que aun no habia corregido los customers ya que hay duplicados
prodcut_to_delete = 118  # Reemplazar con el ID real que hay que eliminar

# Realizar la operación de eliminación
odoo.execute('product.product', 'unlink', [move_to_delete])


{'jsonrpc': '2.0', 'id': 23, 'error': {'code': 200, 'message': 'Odoo Server Error', 'data': {'name': 'odoo.exceptions.MissingError', 'debug': 'Traceback (most recent call last):\n  File "/home/odoo/src/odoo/odoo/http.py", line 1781, in _serve_db\n    return service_model.retrying(self._serve_ir_http, self.env)\n  File "/home/odoo/src/odoo/odoo/service/model.py", line 133, in retrying\n    result = func()\n  File "/home/odoo/src/odoo/odoo/http.py", line 1808, in _serve_ir_http\n    response = self.dispatcher.dispatch(rule.endpoint, args)\n  File "/home/odoo/src/odoo/odoo/http.py", line 2012, in dispatch\n    result = self.request.registry[\'ir.http\']._dispatch(endpoint)\n  File "/home/odoo/src/odoo/addons/website/models/ir_http.py", line 235, in _dispatch\n    response = super()._dispatch(endpoint)\n  File "/home/odoo/src/odoo/odoo/addons/base/models/ir_http.py", line 226, in _dispatch\n    result = endpoint(**request.params)\n  File "/home/odoo/src/odoo/odoo/http.py", line 757, in rou

Exception: Command has failed

In [120]:
# Definir el ID de producto a buscar
product_id_to_delete = 118

# Búsqueda de productos en el modelo 'stock.move'
inventory = odoo.search_read('stock.move', [], ['product_id'], limit=None)

# Convertir los resultados a un DataFrame para una mejor visualización
inventory_df = pd.DataFrame(inventory)

# Extraer el ID del campo 'product_id', que es una lista [ID, Nombre]
# Crear una nueva columna en el DataFrame que contenga solo el número de product_id
inventory_df['product_id_number'] = inventory_df['product_id'].apply(lambda x: x[0] if isinstance(x, list) else None)

# Buscar el registro que tenga el product_id igual a 118
result = inventory_df.loc[inventory_df['product_id_number'] == product_id_to_delete]

# Mostrar los resultados encontrados
if not result.empty:
    print(f"Se encontró el producto con ID {product_id_to_delete} en 'el modelo':\n")
    print(result)
else:
    print(f"No se encontró el producto con ID {product_id_to_delete} en 'el modelo'.")


No se encontró el producto con ID 118 en 'el modelo'.


In [66]:
#hacer el llamado a res.users para mas adelante poder insertar los line items
# Especificar los campos que deseas leer
fields_to_read = ['id', 'name']

# Realizar el llamado al modelo account.move.line con un límite de registros
users = odoo.search_read('res.users', [], fields_to_read, limit=1000)

# Convertir los resultados a un DataFrame
u_df = pd.DataFrame(users)
u_df

,id,name
0,15,Chargebee
1,16,Jim Galvez
2,7,Joel Willis
3,6,Marc Demo
4,10,Valentino
5,11,Valentino
6,14,Valentino
7,13,Valentino
8,12,Valentino


In [53]:
# Agrupar por 'customer_id' y contar la cantidad de veces que aparece
grouped_df = order.groupby('id').size().reset_index(name='cantidad')

# Filtrar para mostrar solo aquellos donde 'cantidad' sea mayor a 1
filtered_df = grouped_df[grouped_df['cantidad'] == 1]

# Mostrar el resultado
filtered_df

,id,cantidad
0,185392,1
1,185393,1
2,185394,1
3,185395,1
4,185396,1
...,...,...
2467,187910,1
2468,187911,1
2469,187912,1
2470,187913,1


In [ ]:
# Sumar la columna 'cantidad' del DataFrame filtrado
total_cantidad = filtered_df['cantidad'].sum()

# Mostrar la sumatoria
print(f"La sumatoria total de la columna 'cantidad' es: {total_cantidad}")


In [51]:
order.select_dtypes(include=['int64', 'float64'])

,net_term_days,exchange_rate,total,amount_paid,amount_adjusted,write_off_amount,credits_applied,amount_due,resource_version,amount_to_collect,...,cn_date_adjustment_credit_notes_expanded,cn_total,cn_date_issued_credit_notes_expanded,cn_total_issued_credit_notes_expanded,created_at,attempt,txn_amount_dunning_attempts_expanded,amount_discounts_expanded,discount_percentage,discount_amount_line_item_discounts_expanded
0,0,1.0,8900,8900,0,0,0,0,1724435719786,0,...,NaN,NaN,NaN,NaN,1.722568e+09,1.0,NaN,NaN,NaN,NaN
1,0,1.0,9900,9900,0,0,0,0,1723568155857,0,...,NaN,NaN,NaN,NaN,1.723518e+09,2.0,NaN,2000.0,NaN,2000.0
2,0,1.0,3900,3900,0,0,0,0,1722508742063,0,...,NaN,NaN,NaN,NaN,1.724382e+09,3.0,NaN,NaN,NaN,NaN
3,0,1.0,3900,3900,0,0,0,0,1723218079989,0,...,NaN,NaN,NaN,NaN,1.722568e+09,1.0,NaN,NaN,NaN,NaN
4,0,1.0,23900,23900,0,0,0,0,1722510311847,0,...,NaN,NaN,NaN,NaN,1.723518e+09,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,0,1.0,0,0,0,0,0,0,1725159618918,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8900.0,100.0,NaN
2462,0,1.0,0,0,0,0,0,0,1725159619104,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9900.0,100.0,NaN
2463,0,1.0,0,0,0,0,0,0,1725159619316,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3900.0,100.0,NaN
2455,0,1.0,0,0,0,0,0,0,1725159635973,0,...,NaN,NaN,NaN,NaN,1.724475e+09,1.0,4600.0,7400.0,100.0,NaN


In [360]:
#muestra todos los campos de tipo object ya que entre ellos estan esos campos a desanidar
order.select_dtypes(include=['int64'])


,net_term_days,total,amount_paid,amount_adjusted,write_off_amount,credits_applied,amount_due,resource_version,amount_to_collect,round_off_amount,tax,sub_total,unit_amount,quantity,amount,tax_amount,discount_amount,item_level_discount_amount
0,0,8900,8900,0,0,0,0,1724435719786,0,0,0,8900,8900,1,8900,0,0,0
1,0,9900,9900,0,0,0,0,1723568155857,0,0,0,9900,0,1,0,0,0,0
2,0,3900,3900,0,0,0,0,1722508742063,0,0,0,3900,8900,1,8900,0,0,0
3,0,3900,3900,0,0,0,0,1723218079989,0,0,0,3900,750,4,3000,0,2000,2000
4,0,23900,23900,0,0,0,0,1722510311847,0,0,0,23900,39,100,3900,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,0,0,0,0,0,0,0,1725159618918,0,0,0,0,5900,1,5900,0,0,0
2467,0,0,0,0,0,0,0,1725159619104,0,0,0,0,0,3,0,0,0,0
2468,0,0,0,0,0,0,0,1725159619316,0,0,0,0,901,1,901,0,0,0
2460,0,0,0,0,0,0,0,1725159635973,0,0,0,0,39,100,3900,0,0,0


In [296]:
lines

,analytic_distribution,analytic_distribution_search,analytic_precision,move_id,journal_id,company_id,company_currency_id,move_name,parent_state,date,...,invoice_origin,payment_matched_amount,l10n_pe_group_id,filter_amount,tax_settlement_move_id,tax_state,subscription_id,subscription_mrr,move_line_ids,withholding_id
string,Analytic Distribution,Analytic Distribution Search,Analytic Precision,Journal Entry,Journal,Company,Company Currency,Number,Status,Date,...,Origin,Payment Matched Amount,Group,Filter Amount,Tax Settlement Move,Tax State,Subscription,Monthly Recurring Revenue,Related Stock Moves,Withholding
type,json,json,integer,many2one,many2one,many2one,many2one,char,selection,date,...,char,monetary,many2one,float,many2one,selection,many2one,monetary,many2many,many2one
searchable,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#campo label es de odoo es equivalente al campo description de line items de chargbee